# 1. Shortest Path in a Grid with Obstacles Elimination

You are given an m x n integer matrix grid where each cell is either 0 (empty) or 1 (obstacle). You can move up, down, left, or right from and to an empty cell in one step.

Return the minimum number of steps to walk from the upper left corner (0, 0) to the lower right corner (m - 1, n - 1) given that you can eliminate at most k obstacles. If it is not possible to find such walk return -1.

![img](https://assets.leetcode.com/uploads/2021/09/30/short1-grid.jpg)

Example 1:
```
Input: grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]], k = 1
Output: 6
Explanation: 
The shortest path without eliminating any obstacle is 10.
The shortest path with one obstacle elimination at position (3,2) is 6. Such path is (0,0) -> (0,1) -> (0,2) -> (1,2) -> (2,2) -> (3,2) -> (4,2).
```

Overview

Like many grid search problems where the goal is to find the shortest path, the key to solve this problem is to apply the Breadth-First Search algorithm, as opposed to the Depth-First Search (DFS) algorithm. In this article, we will start with a classic BFS solution. Then on top of BFS, we will introduce a heuristic (greedy) strategy to speed up the algorithm, which eventually transforms our classic BFS algorithm into another classic algorithm called the A* search algorithm.

Approach 1: BFS (Breadth-First Search)

Intuition

In this problem, we must traverse the grid to reach a target cell, while the grid contains some obstacles. If the problem ends here, one could probably tell that this is a classic grid search problem, e.g. the problem of 1730. Shortest Path to Get Food fits the bill exactly.

However, the particularity of this problem is that one can eliminate obstacles to a certain extent. This constraint complicates our problem. First of all, if there were no limit on how many obstacles we can eliminate, then the shortest distance to reach the target cell would be the Manhattan distance between the starting cell and the target cell. Likewise, if the quota to eliminate the obstacles is greater than the Manhattan distance, then the shortest distance is guaranteed to be the Manhattan distance. However, we do have a limit on the number of obstacles that we can eliminate along the way. As a result, rather than taking a straightforward path to reach the target, we have to take some detour in certain cases, which implies that we need to explore all possible directions while respecting the constraint.

By exploring, we refer to the BFS strategy, rather than DFS. The BFS algorithm works like detecting an object with sonar. A sound wave propagages in all directions with equal speed. At any given moment, all the objects that the sound wave reaches have the same distance to the source of the sound. On the other hand, as soon as the sound wave reaches the object, the path is guaranteed to be the shortest, since the distance is proportional to the time, the more time it takes, the longer the distance is.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_bfs.png)

Given the above intuition of the BFS algorithm, we can rest assured that as soon as we reach the target, the path that leads to the target is the shortest. This is also the rationale for why we should employ the BFS strategy rather than DFS.

Algorithm

In the canonical implementation of the BFS algorithm, we would employ a queue data structure to maintain the order of exploration. Each element in the queue normally contains two pieces of information: the current position and the distance traveled so far from the starting point.

However, in our problem here, we need another piece of information, which is the remaining quota that we can use to eliminate the obstacles.

Together with the coordinates, the obstacle elimination quota uniquely constitutes a state during our BFS exploration. For example, in the following graph, we demonstrate two different paths between the start cell and an intermediate cell.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_bfs_two_paths.png)

Without the obstacle elimination quota, we would only visit the intermediate cell once, while only one of the two paths can lead us to the target, since in one of the paths we don't have sufficient quota to get through. Therefore, it is critical to keep the quota information, so that we can revisit the same cell with different quotas.

Implementation

There are several ways to implement a BFS algorithm. We provide a template to do so in the Queue and Stack Explore Card.

In this section, we provide some sample implementations together with the tweak we mentioned in the above section.

We can break down the algorithm into the following steps:

1. The main body of the BFS algorithm consists of a loop around a queue, as well as a set called seen which keeps track of all the states visited along the way so that we don't visit the same state twice. A state refers to a unique combination of coordinates and the remaining quota.

2. At each iteration of the loop, we pop out one element from the queue. The element contains the distance from the starting point as well as the current state which includes the current coordinate and the remaining quotas to eliminate obstacles.

3. Within the same iteration, we evaluate the next moves starting from the popped element. Each move results in a new state and the state is valid if it is within the grid boundaries and has not been visited before. Each valid state is pushed into the queue for later iterations. Meanwhile, we also need to mark the state as visited by adding the state to the seen states set.

4. At any iteration, if we have reached our target, we can return immediately from the loop.

Note: before running the BFS traversal, we also perform a quick check to see if we have sufficient quotas to take the most direct path from start to finish regardless of the number of obstacles. If so, the shortest distance to reach the target is the Manhattan distance between the starting cell and the target cell.


In [3]:
from collections import deque
class Solution:
    def shortestPath(self, grid, k):
        rows, cols = len(grid), len(grid[0])

        target = (rows-1, cols-1)

        if k >= rows+cols-2:
            return rows+cols-2


        # (row, col, remaining quota)
        state = (0, 0, k)

        queue = deque([(0, state)])

        seen = set([state])

        while queue:
            steps, (row, col, k) = queue.popleft()

            if (row, col) == target:
                return steps
            
            for new_row, new_col in [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]:
                if (0<=new_row<rows) and (0<=new_col<cols):
                    new_elimination = k - grid[new_row][new_col]
                    new_state = (new_row, new_col, new_elimination)

                    if new_elimination>=0 and new_state not in seen:
                        seen.add(new_state)
                        queue.append((steps+1, new_state))
        return -1

# Driver Code
if __name__ == "__main__":
    grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]]
    k = 1
    print(Solution().shortestPath(grid, k))





6


Complexity Analysis

Let NN be the number of cells in the grid, and KK be the quota to eliminate obstacles.

Time Complexity: O(N \cdot K)O(N⋅K)

We conduct a BFS traversal in the grid. In the worst case, we will visit each cell in the grid. And for each cell, at most, it will be visited KK times, with different quotas of obstacle elimination.

Thus, the overall time complexity of the algorithm is O(N \cdot K)O(N⋅K).

Space Complexity: O(N \cdot K)O(N⋅K)

We used a queue to maintain the order of visited states. In the worst case, the queue will contain the majority of the possible states that we need to visit, which in total is N \cdot KN⋅K as we discussed in the time complexity analysis. Thus, the space complexity of the queue is O(N \cdot K)O(N⋅K).

Other than the queue, we also used a set variable (named seen) to keep track of all the visited states along the way. Same as the queue, the space complexity of this set is also O(N \cdot K)O(N⋅K).

To sum up, the overall space complexity of the algorithm is O(N \cdot K)O(N⋅K).

Approach 2: A* (A Star) Algorithm

Intuition

In the above BFS approach, one might notice that when at any specific position, we would systematically explore the surrounding neighbors in all four directions, due to the nature of BFS.

However, the action might seem conterintuitive or sub-optimal. Since the destination is located in the lower-right corner of the grid, in order to find the shortest path, the optimal directions to explore should be either right or down, rather than left or up.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_next_steps.png)

As depicted in the above image, the optimal steps to explore should be the ones in green (right and down), rather than the ones in orange.

Therefore, one idea to improve our BFS approach is to prioritize exploring the most promising directions at each step. Through prioritization, we can speed up the algorithm by reducing the time spent exploring less promising paths.

This idea leads us to the A* search algorithm, which is yet another classic path finding algorithm that uses a heuristic.

Note: we cannot exclude (or prune) those less promising directions, otherwise we might miss the correct path because sometimes we have to take a detour in order to reach the destination.

Algorithm

A* (pronounced as A star) is also known as an informed search algorithm or best-first search. Because at each step of exploration, it makes the best and informed decision on the next steps, i.e. it prioritizes the steps that are the most promising. Specifically, this prioritization strategy can be expressed as A* selects a path that minimizes the following function:f(n) = g(n) + h(n)f(n)=g(n)+h(n)

nn: a specific step during the exploration.

g(n)g(n): the cost to reach the step nn. Here, the cost refers to the distance traveled so far to the step nn.

h(n)h(n): a heuristic estimation on the cost to reach the destination from the step nn. Here, the cost refers to the distance ahead.

f(n)f(n): the estimated total cost to reach the destination if one takes the step nn.

With the defined function, A* algorithm has a deterministic way to evaluate each potential step, and then make what it believes to be the best decision at each step.

The problem boils down to defining the above functions for our scenario, in order to apply the A* algorithm. Specifically, g(n)g(n) would be the number of steps that one has taken to reach nn. And h(n)h(n) would be the Manhattan distance from nn to the destination, which is the shortest path to reach the destination.

The most important property of our heuristic h(n)h(n) function is that the function should be admissible, i.e. it never overestimates the cost. Otherwise, it could not guarantee that the path we find is the shortest one.

To understand the admissible property, let us take a metaphor. In a football tournament, we want to select the best team in the end. If we overestimate the incompetence of a team, i.e. we downplay the potential of the team, we might prematurely disqualify or ignore the team. As a result, we may predict that a not-so-good team will win the championship, while in reality, the team we disqualified or ignored happens to be the best team.

Implementation

The A* algorithm provides a more optimized path selection strategy, on top of the BFS approach. Therefore, we can implement the A* algorithm while keeping the bulk of our previous BFS approach intact. We will still use a queue to keep track of the order of visits. And we will still use a set to keep track of the visited states so that we do not revisit any previously explored paths.

Additionally, here are the modifications that we will make:

* Rather than using a normal queue, we use a priority queue to store the order of visits. The order of visits is based on the estimated total cost function f(n)f(n) that we defined. With the priority queue, the step that has potentially the lowest cost will be visited first.

* For each element in the queue, we add one more piece of information which is the estimated total cost to reach the destination at each step. This estimation will be used to prioritize each potential next step.

* We add another heuristic condition that allows us to determine the length of the shortest path without exploration. The condition is as follows:

    * At any step, if the remaining quota to eliminate the obstacles is larger than the length of the estimated shortest path (i.e. manhattan distance between the current step and the destination), then the length of the remaining path is the manhattan distance.

    * The condition can also be interpreted as if we have sufficient capacity to remove any obstacle along the way, we will simply take the shortest path to reach the destination, without the need for further exploration.

    * We apply the condition at the beginning of each iteration of the loop.


Complexity Analysis

Let NN be the number of cells in the grid, and KK be the quota to eliminate obstacles.

* Time Complexity: O\big(N \cdot K \cdot \log{(N \cdot K)} \big)O(N⋅K⋅log(N⋅K))

We conduct a BFS traversal in the grid. In the worst case, we will visit each cell in the grid. And each cell can be visited at most KK times, with different quotas of obstacle elimination. Therefore, the total number of visits would be N \cdot KN⋅K.

For each visit, we perform one push and one pop operation in the priority queue, which takes O\big(\log{(N \cdot K)} \big)O(log(N⋅K)) time.

Thus, the overall time complexity of the algorithm is O\big(N \cdot K \cdot \log{(N \cdot K)} \big)O(N⋅K⋅log(N⋅K)).

Although the upper bound for the time complexity of the this algorithm is higher than the previous BFS approach, on average, the A* algorithm will outperform the previous BFS approach when there exists any relatively direct path from the source to the target.

* Space Complexity: O(N \cdot K)O(N⋅K)

We use a queue to maintain the order of visited states. In the worst case, the queue could contain the majority of the possible states that we must visit, which in total is N \cdot KN⋅K, as we discussed in the time complexity analysis. Thus, the space complexity of the queue is O(N \cdot K)O(N⋅K).

Other than the queue, we also used a set variable (named seen) to keep track of all the states we visited along the way. Again, the space complexity of this set is also O(N \cdot K)O(N⋅K).

To sum up, the overall space complexity of the algorithm is O(N \cdot K)O(N⋅K).






In [6]:
import heapq
class Solution:
    def shortestPath(self, grid, k):

        rows, cols = len(grid), len(grid[0])
        target = (rows - 1, cols - 1)

        def manhattan_distance(row, col):
            return target[0] - row + target[1] - col

        # (row, col, remaining_elimination)
        state = (0, 0, k)

        # (estimation, steps, state)
        # h(n) = manhattan distance,  g(n) = 0
        queue = [(manhattan_distance(0, 0), 0, state)]
        seen = set([state])

        while queue:
            estimation, steps, (row, col, remain_eliminations) = heapq.heappop(queue)

            # we can reach the target in the shortest path (manhattan distance),
            #   even if the remaining steps are all obstacles
            remain_min_distance = estimation - steps
            if remain_min_distance <= remain_eliminations:
                return estimation

            # explore the four directions in the next step
            for new_row, new_col in [(row, col + 1), (row + 1, col), (row, col - 1), (row - 1, col)]:
                # if (new_row, new_col) is within the grid boundaries
                if (0 <= new_row < rows) and (0 <= new_col < cols):
                    new_eliminations = remain_eliminations - grid[new_row][new_col]
                    new_state = (new_row, new_col, new_eliminations)

                    # if the next direction is worth exploring
                    if new_eliminations >= 0 and new_state not in seen:
                        seen.add(new_state)
                        new_estimation = manhattan_distance(new_row, new_col) + steps + 1
                        heapq.heappush(queue, (new_estimation, steps + 1, new_state))

        # did not reach the target
        return -1

# Driver Code
if __name__ == "__main__":
    grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]]
    k = 1
    print(Solution().shortestPath(grid, k))

6


# 2. Step-By-Step Directions From a Binary Tree Node to Another

You are given the root of a binary tree with n nodes. Each node is uniquely assigned a value from 1 to n. You are also given an integer startValue representing the value of the start node s, and a different integer destValue representing the value of the destination node t.

Find the shortest path starting from node s and ending at node t. Generate step-by-step directions of such path as a string consisting of only the uppercase letters 'L', 'R', and 'U'. Each letter indicates a specific direction:

* 'L' means to go from a node to its left child node.
* 'R' means to go from a node to its right child node.
* 'U' means to go from a node to its parent node.

Return the step-by-step directions of the shortest path from node s to node t.

Example 1:
![img](https://assets.leetcode.com/uploads/2021/11/15/eg1.png)
```
Input: root = [5,1,2,3,null,6,4], startValue = 3, destValue = 6
Output: "UURL"
Explanation: The shortest path is: 3 → 1 → 5 → 2 → 6.
```

Example 2:
![img](https://assets.leetcode.com/uploads/2021/11/15/eg2.png)
```
Input: root = [2,1], startValue = 2, destValue = 1
Output: "L"
Explanation: The shortest path is: 2 → 1.
```

## Approach 1

* Build directions for both start and destination from the root.
    * Say we get "LLRRL" and "LRR".

* Remove common prefix path.
    * We remove "L", and now start direction is "LRRL", and destination - "RR"

* Replace all steps in the start direction to "U" and add destination direction.
    * The result is "UUUU" + "RR".

In [9]:
class Node:
    def __init__(self, value):
        self.val = value
        self.left, self.right = None, None

class Solution:
    def getDirections(self, root, startValue, destValue):
        def find(n, value, path):
            if n.val == value:
                return True
            if n.left and find(n.left, value, path):
                path+= "L"
            elif n.right and find(n.right, value, path):
                path+= "R"
            return path 
        s, d = [], []
        s = find(root, startValue, s)
        d = find(root, destValue, d)

        while len(s) and len(d) and s[-1]==d[-1]:
            s.pop()
            d.pop()
        return "".join("U" * len(s)) + "".join(reversed(d))

# Driver Code
if __name__ == "__main__":
    root = Node(5)
    root.left = Node(1)
    root.left.left = Node(3)
    root.right = Node(2)
    root.right.left = Node(6)
    root.right.right = Node(4)
    startValue = 3
    destValue = 6
    print(Solution().getDirections(root, startValue, destValue))


UURL


## Approach 2: LCA


In [10]:
class Solution:
    def getDirections(self, root, startValue, destValue):
        
        def lca(node): 
            """Return lowest common ancestor of start and dest nodes."""
            if not node or node.val in (startValue , destValue): return node 
            left, right = lca(node.left), lca(node.right)
            return node if left and right else left or right
        
        root = lca(root) # only this sub-tree matters
        
        ps = pd = ""
        stack = [(root, "")]
        while stack: 
            node, path = stack.pop()
            if node.val == startValue: ps = path 
            if node.val == destValue: pd = path
            if node.left: stack.append((node.left, path + "L"))
            if node.right: stack.append((node.right, path + "R"))
        return "U"*len(ps) + pd

# Driver Code
if __name__ == "__main__":
    root = Node(5)
    root.left = Node(1)
    root.left.left = Node(3)
    root.right = Node(2)
    root.right.left = Node(6)
    root.right.right = Node(4)
    startValue = 3
    destValue = 6
    print(Solution().getDirections(root, startValue, destValue))

UURL


# 3. Race Car

Your car starts at position 0 and speed +1 on an infinite number line. Your car can go into negative positions. Your car drives automatically according to a sequence of instructions 'A' (accelerate) and 'R' (reverse):

* When you get an instruction 'A', your car does the following:
```python
position += speed
speed *= 2
```
* When you get an instruction 'R', your car does the following:
```python
If your speed is positive then speed = -1
otherwise speed = 1
```
Your position stays the same.

For example, after commands "AAR", your car goes to positions 0 --> 1 --> 3 --> 3, and your speed goes to 1 --> 2 --> 4 --> -1.

Given a target position target, return the length of the shortest sequence of instructions to get there.

 
```
Example 1:

Input: target = 3
Output: 2
Explanation: 
The shortest instruction sequence is "AA".
Your position goes from 0 --> 1 --> 3.
Example 2:

Input: target = 6
Output: 5
Explanation: 
The shortest instruction sequence is "AAARA".
Your position goes from 0 --> 1 --> 3 --> 7 --> 7 --> 6.
```

I -- BFS solution

Well, the BFS solution is straightforward: we can keep track of all the possible positions of the racecar after n instructions (n = 0, 1, 2, 3, 4, ...) and return the smallest n such that the target position is reached. Naive BFS will run at O(2^n) since for each position we have two choices: either accelerate or reverse. Further observations reveal that there may be overlapping among intermediate states so we need to memorize visited states (each state is characterized by two integers: car position and car speed). However, the total number of unique states still blows up for large target positions (because the position and speed can grow unbounded), so we need further pruning of the search space.

II -- DP solution

DP solution works by noting that after each reverse, the car's speed returns to 1 (the sign can be interpreted as the direction of the speed). So we can redefine the problem in terms of the position of the car while leave out the speed: let T(i) be the length of the shortest instructions to move the car from position 0 to position i, with initail speed of 1 and its direction pointing towards position i. Then our original problem will be T(target), and the base case is T(0) = 0. Next we need to figure out the recurrence relations for T(i).

Note that to apply the definition of T(i) to subproblems, the car has to start with speed of 1, which implies we can only apply T(i) right after the reverse instruction. Also we need to make sure the direction of the initial speed when applying T(i) is pointing towards the final target position.

However, we don't really know how many accelerate instructions there should be before the reverse instruction, so theoretically we need to try all possible cases: zero A, one A, two A, three A, ... and so on. For each case, we can obtain the position of the car right before the reverse instruction, which will be denoted as j = 2^m - 1, with m the number of A's. Then depending on the relation between i and j, there will be three cases:

j < i: the reverse instruction is issued before the car reaches i. In this case, we cannot apply the definition of T(i) to the subproblems directly, because even though the speed of the car returns to 1, its direction is pointing away from the target position (in this case position i). So we have to wait until the second reverse instruction is issued. Again, we don't really know how many accelerate instructions there should be in between these two reverse instructions, so we will try each of the cases: zero A, one A, two A, three A, ..., etc. Assume the number of A is q, then the car will end up at position j - p right before the second reverse instruction, where p = 2^q - 1. Then after the second reverse instruction, our car will start from position j - p with speed of 1 and its direction pointing towards our target position i. Since we want the length of the total instruction sequence to be minimized, we certainly wish to use minimum number of instructions to move the car from j - p to i, which by definition will be given by T(i-(j-p)) (note that in the definition of T(i), we move the car from position 0 to position i. If the start position is not 0, we need to shift both the start and target positions so that the start position is aligned with 0). So in summary, for this case, the total length of the instruction will be given by: m + 1 + q + 1 + T(i-(j-p)), where m is the number of A before the first R, q is the number of A before the second R, the two 1's correspond to the two R's, and lastly T(i-(j-p)) is the length of instructions moving the car from position j - p to the target position i.

j == i: the target position is reached without any reverse instructions. For this case, the total length of the instruction will be given by m.

j > i: the reverse instruction is issued after the car goes beyond i. In this case, we don't need to wait for a second reverse instruction, because after the first reverse instruction, the car's speed returns to 1 and its direction will be pointing towards our target position i. So we can apply the definition of T(i) directly to the subproblem, which will be T(j-i). Note that not only do we need to shift the start and target positions, but also need to swap them as well as the directions. So for this case, the total length of the instructions will be given by m + 1 + T(j-i).

Our final answer for T(i) will be the minimum of the above three cases.

**Time and Space Complexity:**
Both the top-down DP and bottom-up DP run at O(target * (log(target))^2) with O(target) space. However, the top-down DP may be slightly more efficient as it may skip some of the intermediate cases that must be computed explicitly for the bottom-up DP. Though the nominal time complexity are the same, both DP solutions will be much more efficient in practice compared to the BFS solution, which has to deal with (position, speed) pairs and their keys for hashing, etc.

In [14]:
# bfs 
import collections
class Solution:
    def racecar(self, target: int) -> int:
        
        #1. Initialize double ended queue as 0 moves, 0 position, +1 velocity
        queue = collections.deque([(0, 0, 1)])
        while queue:
            
            # (moves) moves, (pos) position, (vel) velocity)
            moves, pos, vel = queue.popleft()

            if pos == target:
                return moves
            
            #2. Always consider moving the car in the direction it is already going
            queue.append((moves + 1, pos + vel, 2 * vel))
            
            #3. Only consider changing the direction of the car if one of the following conditions is true
            #   i.  The car is driving away from the target.
            #   ii. The car will pass the target in the next move.  
            if (pos + vel > target and vel > 0) or (pos + vel < target and vel < 0):
                queue.append((moves + 1, pos, -vel / abs(vel)))

# Driver Code
if __name__ == "__main__":
    target = 3
    print(Solution().racecar(target))   

2


In [12]:
# dp solution
class Solution:
    def __init__(self):
        self.dp = {0: 0}
    def racecar(self, t):
        if t in self.dp:
            return self.dp[t]
        n = t.bit_length()  
        print(f"n = {n}")
        if 2**n - 1 == t:
            self.dp[t] = n
        else:
            self.dp[t] = self.racecar(2**n - 1 - t) + n + 1
            for m in range(n - 1):
                self.dp[t] = min(self.dp[t], self.racecar(t - 2**(n - 1) + 2**m) + n + m + 1)
        return self.dp[t]

# Driver Code
if __name__ == "__main__":
    t = 3
    print(Solution().racecar(t))


n = 2
2


# 4. Find Leaves of Binary Tree

Given the root of a binary tree, collect a tree's nodes as if you were doing this:

Collect all the leaf nodes.
Remove all the leaf nodes.
Repeat until the tree is empty.
 

Example 1:
![img](https://assets.leetcode.com/uploads/2021/03/16/remleaves-tree.jpg)
```
Input: root = [1,2,3,4,5]
Output: [[4,5,3],[2],[1]]
Explanation:
[[3,5,4],[2],[1]] and [[3,4,5],[2],[1]] are also considered correct answers since per each level it does not matter the order on which elements are returned.
Example 2:

Input: root = [1]
Output: [[1]]
```

Approach 1: DFS (Depth-First Search) with sorting

Intuition

The order in which the elements (nodes) will be collected in the final answer depends on the "height" of these nodes. The height of a node is the number of edges from the node to the deepest leaf. The nodes that are located in the ith height will be appear in the ith collection in the final answer. For any given node in the binary tree, the height is obtained by adding 1 to the maximum height of any children. Formally, for a given node of the binary tree \text{root}root, it's height can be represented as

$$\text{height(root)} = \text{1} + \text{max(height(root.left), height(root.right))}$$

Where $\text{root.left}$ and $\text{root.right}$ are left and right children of the root respectively

Algorithm

In our first approach, we'll simply traverse the tree recursively in a depth first search manner using the function int getHeight(node), which will return the height of the given node in the binary tree. Since height of any node depends on the height of it's children node, hence we traverse the tree in a post-order manner (i.e. height of the childrens are calculated first before calculating the height of the given node). Additionally, whenever we encounter a null node, we simply return -1 as it's height.

Next, we'll store the pair (height, val) for all the nodes which will be sorted later to obtain the final answer. The sorting will be done in increasing order considering the height first and then the val. Hence we'll obtain all the pairs in the increasing order of their height in the given binary tree.

Complexity Analysis

Time Complexity: Assuming NN is the total number of nodes in the binary tree, traversing the tree takes O(N)O(N) time. Sorting all the pairs based on their height takes O(N \log N)O(NlogN) time. Hence overall time complexity of this approach is O(N \log N)O(NlogN)

Space Complexity: O(N)O(N), the space used by pairs. solution also requires O(N)O(N) space however the output does not count towards the space complexity.


In [25]:
class Solution:
    def __init__(self):
        self.pairs = []

    def getHeight(self, root):
        if not root:
            return -1
        leftHeight = self.getHeight(root.left)
        rightHeight = self.getHeight(root.right)

        currHeight = max(leftHeight, rightHeight) + 1

        self.pairs.append((currHeight, root.val))
        return currHeight
    
    def findLeaves(self, root):
        self.getHeight(root)
        self.pairs.sort(key=lambda x: x[0])
        # print(self.pairs)
        n = len(self.pairs)
        height = 0
        i=0
        solution = []
        while i<n:
            nums = []
            while i<n and self.pairs[i][0] == height:
                nums.append(self.pairs[i][1])
                i += 1
            solution.append(nums)
            # print(solution)
            height += 1
        return solution

# Driver Code
if __name__ == "__main__":
    root = Node(1)
    root.left = Node(2)
    root.right = Node(3)
    root.left.left = Node(4)
    root.left.right = Node(5)
    print(Solution().findLeaves(root))


[[4, 5, 3], [2], [1]]


# 5.  Remove All Ones With Row and Column Flips

You are given an m x n binary matrix grid.

In one operation, you can choose any row or column and flip each value in that row or column (i.e., changing all 0's to 1's, and all 1's to 0's).

Return true if it is possible to remove all 1's from grid using any number of operations or false otherwise.

Example 1:
![img](https://assets.leetcode.com/uploads/2022/01/03/image-20220103191300-1.png)
```
Input: grid = [[0,1,0],[1,0,1],[0,1,0]]
Output: true
Explanation: One possible way to remove all 1's from grid is to:
- Flip the middle row
- Flip the middle column
```

Explanation
```
I honestly don't know how to categorize this problem. It seems like a Math problem to me. Once you understand the logic, the implementation is simple.
Basically the "pattern" of each row should be the same, by pattern, I mean following:
001100 and 001100 are the same pattern
001100 and 110011 (the invert of original) are the same pattern
Only in above situation, one matrix can be converted to all zero
Intuition?
Believe it or not, I draw a couple examples to test it out and suddenly it becomes obvious
I guess it's good habit to get your hands dirty :)
```



In [31]:
class Solution:
    def removeOnes(self, grid):
        r1, r1_invert = grid[0], [1-val for val in grid[0]]
        for i in range(1, len(grid)):
            if grid[i] != r1 and grid[i] != r1_invert:
                return False
        return True

# Driver Code
if __name__ == "__main__":
    grid = [[0,1,0],[1,0,1],[0,1,0]]
    print(Solution().removeOnes(grid))

True


In [30]:
class Solution:
    def removeOnes(self, grid):
        return all(grid[i] == grid[0] or grid[i] == [1-val for val in grid[0]] for i in range(len(grid)))

# Driver Code
if __name__ == "__main__":
    grid = [[0,1,0],[1,0,1],[0,1,0]]
    print(Solution().removeOnes(grid))

True


# 6. Evaluate Reverse Polish Notation

Evaluate the value of an arithmetic expression in Reverse Polish Notation.

Valid operators are +, -, *, and /. Each operand may be an integer or another expression.

Note that division between two integers should truncate toward zero.

It is guaranteed that the given RPN expression is always valid. That means the expression would always evaluate to a result, and there will not be any division by zero operation.
```
Example 1:

Input: tokens = ["2","1","+","3","*"]
Output: 9
Explanation: ((2 + 1) * 3) = 9
Example 2:

Input: tokens = ["4","13","5","/","+"]
Output: 6
Explanation: (4 + (13 / 5)) = 6
Example 3:

Input: tokens = ["10","6","9","3","+","-11","*","/","*","17","+","5","+"]
Output: 22
Explanation: ((10 * (6 / ((9 + 3) * -11))) + 17) + 5
= ((10 * (6 / (12 * -11))) + 17) + 5
= ((10 * (6 / -132)) + 17) + 5
= ((10 * 0) + 17) + 5
= (0 + 17) + 5
= 17 + 5
= 22
```

If you've attempted this question and can't figure out why you're getting wrong answers, here are a couple of things to check before reading the article:

Reverse Polish Notation is not a "reverse" form of Polish Notation. It is a bit different.

If you're using Java, note that the input type is an array of strings, not an array of chars. This means that you should be comparing them with .equals(...), not ==. If your code is working on your computer but not on Leetcode, this is probably why. It is a bug in your code, not in the Leetcode platform.

Some programming languages (e.g. Python, but not C++ and Java) do not truncate towards 0 with division, so you'll need to figure out how to make them do so (we'll discuss ways in the article). For example, if we put -121 // 7 into Python, we get -18, but we actually wanted -17. If unsure about your programming language, either check the documentation or simply write a program that does -121 / 7 (as integer division) and see which result you get.

Did you put numbers around the correct way? e.g. the test case ["12", "7", "-"] means you should calculate 12 - 7 = 5, and not 7 - 12 = -5. In most implementations, getting them the correct way around is not immediately obvious. If you aren't certain you have it right, try this test case (["12", "7", "-]) and check whether you get 5 or -5.

At the time of writing this solution article, the Wikipedia article has a number of errors and sections that are confusing (in particular, parsing the list in reverse). Try to understand how Reverse Polish Notation works and then design an algorithm yourself rather than following the provided pseudocode too closely. The Introduction section of this solution article also explains how Reverse Polish Notation works.

**Introduction**

We'll start with looking at what it means for integer division to truncate towards zero, and then at what Reverse Polish Notation is. 

Division between two integers should truncate towards zero

Early on in your programming career, you probably learned about integer division. When dividing 2 positive numbers, we always truncate down to the nearest integer. The non-integer values are in parenthesis afterwards for reference.
```
6 / 2 = 3 (3.0)
11 / 5 = 2 (2.2)
9 / 5 = 1 (1.8)
```

Most programming languages do integer division by default (as opposed to float division, where decimal places are kept), and this is how they handle positive integers. Both of the following definitions could be (and are) used to describe the truncation.

* The result is truncated to a less than or equal number. i.e. 1 is less than 1.8.
* The truncation is towards zero, i.e. 1 is closer to zero than 1.8 is.

For negative numbers however, it is impossible to satisfy both of these, so one or the other has to be picked. For example, consider the following:
```
-9 / 5 = ? (-1.8)
```

* If we wanted the truncated result to be smaller, we'd have to go to -2, as -2 < -1.
* If we wanted the truncated result to be nearer to zero, we'd have to go to -1 as -1 is nearer to zero than -2 is.
Some programming languages go with the first definition, and others go with the second. For this problem, you are expected to go with the second definition, regardless of what your chosen programming language uses.

Python, for example, goes with the first definition. This means that we need to find a way of doing the division. Luckily, the int(...) function does truncate towards zero, and therefore we can use the int(a / b) trick. Note that this is not the same as int(a // b). We haven't checked what all the programming languages available on Leetcode do, so if your chosen programming language is not truncating division towards zero, have a look in the math libraries for your chosen programming language or do a web search.

**What is Infix Notation?**

Analysing Infix Notation provides some great context for understanding Reverse Polish Notation.

Most people know how to read expressions written using Infix Notation. Evaluating the following expression is something you will have learned to do in elementary school.
```
3 + 1 + 9 - 5 = 8
```
This isn't too difficult. However, many of you will also have seen viral posts circulating social media websites, such as Facebook, that challenge you to evaluate an expression like:
```
5 * 4 + 9 - 2 / 3 + 1 = ?
```
When you check the comments, among many other strange answers, you'll probably see people arguing about whether the answer is 10, 27.33, or 29.33.

The reason for the disagreement is because different people have different understandings about how such an expression should be evaluated.

Those who say the answer is 10 evaluated it strictly from left to right, with the following steps:
```
5 * 4 = 20
20 + 9 = 29
29 - 2 = 27
27 / 3 = 9
9 + 1 = 10
```
Those who say the answer is 27.33 follow a rule where we evaluate operations in the following order; division, multiplication, addition, and subtraction. This method comes from a common misunderstanding of the widely used mnemonics: PEMDAS/BODMAS/BEDMAS. The steps with this method are as follows:
```python
= 5 * 4 + 9 - 0.66 + 1 (Do division first.)
= 20 + 9 - 0.66 + 1 (Do multiplication second.)
= 29 - 1.66 (Do the additions third.)
= 27.33 (Do the subtraction fourth.)
```
Those who say the answer is 29.33 use the rules most programming languages use, and that is also the correct interpretation of the mnemonics (PEMDAS/BODMAS/BEDMAS). That is to do division and multiplication first, in order from left to right, and then addition and subtraction, in order from left to right. Their steps are as follows:
```python
= 20 + 9 - 2 / 3 + 1 (Do the multiplication.)
= 20 + 9 - 0.66 + 1 (Do the division.)
= 29 - 0.66 + 1 (Do the first addition.)
= 28.33 + 1 (Do the subtraction, as it's next.)
= 29.33 (Do the last addition.)
```
Most mathematicians would agree that the correct answer is 29.33. Yet, this is probably not the answer you'd get if you asked a random sample of people in the general public (just look at the Facebook posts!). In school for example, I was taught the method that gives 28.33. It wasn't until I learned programming in university that I learned the correct way!

When we want to do the operations in a different order, we use parenthesis (brackets) around the parts to do first. The parts in parenthesis are always done before the parts outside.

The big disadvantage of Infix Notation is hopefully clear now. The rules for evaluating it are surprisingly complex, cause a lot of confusion, and in fact most people don't understand them properly. Additionally, the need to use parentheses correctly adds another layer of complexity.

As we move towards understanding what Reverse Polish Notation is, keep in mind that while it seems a bit strange and un-intuitive (at first!), that Infix Notation is actually more confusing. The only reason Infix Notation seems intuitive is because you've probably been using it all your life and so it is now second nature to you. People who use Reverse Polish Notation on a daily basis find it very intuitive! Some hand-held calculators still use it!

What is Reverse Polish Notation?

Just like Infix Notation, or in fact any other notation, Reverse Polish Notation has rules for how to evaluate it. You'll need to know these rules before you can write an algorithm. The rules could either be prior knowledge or supplied by an interviewer.
```python
While there are operators remaining in the list, find the left-most operator. Apply it to the 2 numbers immediately before it, and replace all 3 tokens (the operator and 2 numbers) with the result.
```

For example in the most simplest case of 3 4 + when we reach + we can replace 3 4 + with it's result 7.

As long as the input was valid, this rule will always work and leave a single number that should be returned. The leftmost operator that hasn't yet been removed will always have 2 numbers immediately before it.

Hopefully the advantage is obvious now. Reverse Polish Notation doesn't require brackets, and the rules for evaluating it are far simpler. In-fact, our equivalent question for Infix Notation is much more difficult than this one!

**Approach : Evaluate with Stack**

Intuition

We don't want to repeatedly delete items from the middle of a list, as this inevitably leads to O(n^2) time performance. So recall that the above algorithm scanned through the list from left to right, and each time it reached an operator, it'd replace the operator and the 2 numbers immediately before it with the result of applying the operator to the 2 numbers.

The two key steps of the above algorithm were:

* Visit each operator, in linear order. Finding these can be done with a linear search of the original list.
* Get the 2 most recently seen numbers that haven't yet been replaced. These could be tracked using a Stack.

The algorithm would be as follows:
```python
stack = new Stack()
for each token in tokens:
    if token is a number:
        stack.push(token)
    else (token is operator):
        number_2 = stack.pop()  
        number_1 = stack.pop()
        result = apply_operator(token, number_1, number_2)
        stack.push(result)
return stack.pop()
```

ou might have noticed the following 2 lines of the pseudocode could look like they're around the wrong way.
```
number_2 = stack.pop()  
number_1 = stack.pop()
```
They are correct though. Remember that for division and subtraction, the order of the numbers matters. i.e. 7 - 5 ≠ 5 - 7. On the Stack, we have the second on the top. So we need to reverse them before applying the operator.

Algorithm

Here is code that uses lambda functionality.

**Complexity Analysis**

Let nn be the length of the list.

* Time Complexity : O(n)O(n).

We do a linear search to put all numbers on the stack, and process all operators. Processing an operator requires removing 2 numbers off the stack and replacing them with a single number, which is an O(1)O(1) operation. Therefore, the total cost is proportional to the length of the input array. Unlike before, we're no longer doing expensive deletes from the middle of an Array or List.

* Space Complexity : O(n)O(n).

In the worst case, the stack will have all the numbers on it at the same time. This is never more than half the length of the input array.



In [37]:
def evalRPN(tokens):
    # create a stack
    stack = []

    # operations dict
    operations = {
        "+": lambda a, b: a+b,
        "-": lambda a, b: a-b,
        "/": lambda a, b: int(a/b),
        "*": lambda a, b: a*b,
    }

    for token in tokens:
        if token in operations:
            num2 = stack.pop()
            num1 = stack.pop()
            result = operations[token](num1, num2)
            stack.append(result)
        else:
            stack.append(int(token))
    return stack.pop()

# Driver Code
if __name__ == "__main__":
    tokens = ["2", "1", "+", "3", "*"]
    print(evalRPN(tokens))

9


# 7. Minimum Time Difference

Given a list of 24-hour clock time points in "HH:MM" format, return the minimum minutes difference between any two time-points in the list.
```
Example 1:

Input: timePoints = ["23:59","00:00"]
Output: 1
Example 2:

Input: timePoints = ["00:00","23:59","00:00"]
Output: 0
```

Convert each timestamp to it's integer number of minutes past midnight, and sort the array of minutes.
The required minimum difference must be a difference between two adjacent elements in the circular array (so the last element is "adjacent" to the first.) We take the minimum value of all of them.




In [46]:

def findMinDifference(A):
    def convert(time):
        return int(time[:2]) * 60 + int(time[3:])
    minutes = sorted(convert(time) for time in A)
    print(f"minutes = {minutes}")
    mm = minutes[1:] + minutes[:1]
    print(f"mm = {mm}")
    for x, y in zip(minutes, minutes[1:] + minutes[:1]):
        print(x, y)
    return min( (y - x) % (24 * 60) 
                for x, y in zip(minutes, minutes[1:] + minutes[:1]) )

# Driver Code
if __name__ == "__main__":
    A = ["00:00","23:59","00:00"]
    print(findMinDifference(A))

minutes = [0, 0, 1439]
mm = [0, 1439, 0]
0 0
0 1439
1439 0
0


1. Convert each time to minutes. O(n).
2. Sort the list of minutes. O(nlogn).
3. Calculate the distance from p[i] to p[i+1] for all i except p[n-1] where n is the length of the times array. O(n).
4. Calculate the distance of the final point to it's first clockwise element, which will cross 0. This is the point in which there would be a revolution loop. O(1).

At each step between 3 and 4, minDist = min(current, new).

**Time Complexity**
* Time Complexity : O(nlogn)
* Space Complexity : O(1)

In [48]:
import sys
def toMin(time):
    time = time.split(':')
    res = (60*int(time[0])) + int(time[1])
    return res
    
class Solution:
    def findMinDifference(self, timePoints):
        for i, time in enumerate(timePoints):
            timePoints[i] = toMin(time)
        
        res = sys.maxsize
        timePoints.sort()
        for i in range(0, len(timePoints) - 1): #calculate the closest CW distance of each element except last
            res = min(res, (timePoints[i+1] - timePoints[i]))
        
        res = min(res, 60* 24 - timePoints[-1] + timePoints[0]) #calc final point
        
        return res

# Driver Code
if __name__ == "__main__":
    A = ["00:00","23:59","00:00"]
    print(Solution().findMinDifference(A))


0


# 8. Stock Price Fluctuation

You are given a stream of records about a particular stock. Each record contains a timestamp and the corresponding price of the stock at that timestamp.

Unfortunately due to the volatile nature of the stock market, the records do not come in order. Even worse, some records may be incorrect. Another record with the same timestamp may appear later in the stream correcting the price of the previous wrong record.

Design an algorithm that:

* Updates the price of the stock at a particular timestamp, correcting the price from any previous records at the timestamp.
* Finds the latest price of the stock based on the current records. The latest price is the price at the latest timestamp recorded.
* Finds the maximum price the stock has been based on the current records.
* Finds the minimum price the stock has been based on the current records.

Implement the StockPrice class:

* StockPrice() Initializes the object with no price records.
* void update(int timestamp, int price) Updates the price of the stock at the given timestamp.
* int current() Returns the latest price of the stock.
* int maximum() Returns the maximum price of the stock.
* int minimum() Returns the minimum price of the stock.

```
Example 1:

Input
["StockPrice", "update", "update", "current", "maximum", "update", "maximum", "update", "minimum"]
[[], [1, 10], [2, 5], [], [], [1, 3], [], [4, 2], []]
Output
[null, null, null, 5, 10, null, 5, null, 2]

Explanation
StockPrice stockPrice = new StockPrice();
stockPrice.update(1, 10); // Timestamps are [1] with corresponding prices [10].
stockPrice.update(2, 5);  // Timestamps are [1,2] with corresponding prices [10,5].
stockPrice.current();     // return 5, the latest timestamp is 2 with the price being 5.
stockPrice.maximum();     // return 10, the maximum price is 10 at timestamp 1.
stockPrice.update(1, 3);  // The previous timestamp 1 had the wrong price, so it is updated to 3.
                          // Timestamps are [1,2] with corresponding prices [3,5].
stockPrice.maximum();     // return 5, the maximum price is 5 after the correction.
stockPrice.update(4, 2);  // Timestamps are [1,2,4] with corresponding prices [3,5,2].
stockPrice.minimum();     // return 2, the minimum price is 2 at timestamp 4.
```


**Overview**
We are given a stream of records of the price of stock at different timestamps. We need to implement some functions to get the lowest, highest, and latest price of the stock based on the records provided.

The records do not come in chronological order. Another record with the same timestamp may appear later in the stream, correcting the price of the previous wrong record; this correction can affect the lowest, highest, and latest price of the stock.

Let's walk through an example to make sure we clearly understand what this problem is asking. Suppose the stock prices in increasing order of timestamp are, [2, 10, 3, 3, 5, 9].
Here, the lowest price of the stock is 2, the highest price is 10, and the latest price is 9.

If the $1^{st}$ record is corrected from 2 to 4, then the lowest price changes from 2 to 3.

If the last record is corrected from 9 to 6, then the latest price changes to 6.

If $2^{nd}$ record is corrected from 10 to 20, then the highest price changes to 20.

Thus, we need to store all the records of the stock price and make corrections when necessary.
We can use a hashmap to store timestamp as key and price as value, where the correction of records will only take constant time.

Now we need to get the highest and lowest price of the stock. We could think of sorting the stock prices but when an update or an insertion of a new price is made we would need to sort the prices again.

Thus, we will prefer using a data structure that allows us to track the current minimum and maximum values efficiently. **We can use a sorted set, sorted map, or min and max-heaps; the intuition will remain the same only the implementation will differ.** All these data structures keep the elements sorted, and the insertion and removal of elements take only logarithmic time.

**Approach 1: Hashed and Sorted Map**

Intuition

We will use a hashmap (timestampPriceMap) to store the price of the stocks. A hashmap stores the elements as key-value pairs. Here, our key is the timestamp and the value is the price of the stock at the respective timestamp, i.e. the hashmap maps timestamp to price.

Now, we will use a sorted map (priceFrequency) to store the stock prices in increasing order.
A sorted map also stores the elements as key-value pairs and keeps elements sorted on the basis of the key. Thus, we will store the price of the stocks as the key and its occurrence (count) in our records stream as the value, i.e. the sorted map maps the price to the frequency. It denotes how many times a price is present in our records stream.

Insertion of Record:

* Insert the stock price at the current timestamp in timestampPriceMap.
* Increase the stock price's count in priceFrequency. Initially, it is assumed the count is 0 when the price is not present in the map.

Updation of Record:

* Update the stock price of the current timestamp in timestampPriceMap.
* Decrease the count of the old price from priceFrequency and if the count reaches 00 then remove it and increase the correct stock price's count.

Get the Latest Price:

* Use one variable to keep track of the latest time and get the stock's price at the latest time from timestampPriceMap.

Get Minimum and Maximum Stock Price:

* The first element's key of priceFrequency is the lowest price of the stock.
* The last element's key of priceFrequency is the highest price of the stock.

**Algorithm**

* Initialize variables:
    * latestTime, variable to store the latest timestamp according to the records.
    * timestampPriceMap, a hashmap to store timestamp and prices of the stock.
    * priceFrequency, a sorted map to store all prices in increasing order.
* In the update function, the current record can be a new record or a correction to an old record:
    * Try to update latestTime, to the current timestamp.
    * If the current timestamp is already present in timestampPriceMap it means this record is a correction then we reduce the count of the old price from priceFrequency.
    * Add/update the current timestamp's price in timestampPriceMap.
    * Increment the count of the current timestamp's price in priceFrequency.
* In the current function, we need to return the latest price of the stock, i.e. price of the stock at latestTime in timestampPriceMap.
* In the maximum function, we need to return the maximum stock price, i.e. the key of the last element in priceFrequency's.
* In the minimum function, we need to return the minimum stock price, i.e. the key of the first element in priceFrequency's.

**Complexity Analysis**

If NN is the number of records in the input stream.

* Time complexity: O(N \log N)O(NlogN)

    * In the update function, we add and remove a record in both hashmap and sorted map. In hashmap, both operations take constant time, but in the sorted map they take O(\log N)O(logN) time.

    * Each call to the maximum, minimum, or current function will take only constant time to return the result.

    * In the worst-case scenario, all NN calls will be to the update function, which will require a total of O(N \log N)O(NlogN) time.

* Space complexity: O(N)O(N)

    * In the update function, we add and remove a record in both the hashmap and sorted map. Thus each function call takes O(1)O(1) space. So for NN update calls, it will take O(N)O(N) space.

    * The maximum, minimum, and current functions do not use any additional space.

    * Thus, in the worst-case, we will add all NN records in both the hashmap and sorted map, which takes O(N)O(N) space.





In [49]:
from sortedcontainers import SortedDict

class StockPrice:
    def __init__(self):
        self.latest_time = 0
        # Store price of each stock at each timestamp.
        self.timestamp_price_map = {}
        # Store stock prices in increasing order to get min and max price.
        self.price_frequency = SortedDict()
        
    def update(self, timestamp: int, price: int) -> None:
        # Update latest_time to latest timestamp.
        self.latest_time = max(self.latest_time, timestamp)
        
        # If same timestamp occurs again, previous price was wrong. 
        if timestamp in self.timestamp_price_map:
            # Remove previous price.
            old_price = self.timestamp_price_map[timestamp]
            self.price_frequency[old_price] -= 1
            
            # Remove the entry from the sorted-dictionary.
            if not self.price_frequency[old_price]:
                del self.price_frequency[old_price]
        
        # Add latest price for timestamp.
        self.timestamp_price_map[timestamp] = price
        
        if price in self.price_frequency:
            self.price_frequency[price] += 1
        else:
            self.price_frequency[price] = 1

    def current(self) -> int:
        # Return latest price of the stock.
        return self.timestamp_price_map[self.latest_time]
        
    def maximum(self) -> int:
        # Return the maximum price stored at the end of sorted-dictionary.
        return self.price_frequency.peekitem(-1)[0]
        
    def minimum(self) -> int:
        # Return the maximum price stored at the front of sorted-dictionary.
        return self.price_frequency.peekitem(0)[0]

# Driver Code
if __name__ == "__main__":
    stock = StockPrice()
    stock.update(1, 10)
    stock.update(2, 5)
    print(stock.current())
    print(stock.maximum())
    stock.update(1, 3)
    print(stock.maximum())
    stock.update(4, 2)
    print(stock.minimum())
   

5
10
5
2


## Approach 2: Hashmap and Heaps

Intuition

In this approach, again, we will use a hashmap (timestampPriceMap) to record the stock's price at each timestamp.

However, it is not necessary for us to maintain a sorted map as we did in the previous approach. Any time we need to efficiently keep track of the lowest or highest value, we should consider using a heap data structure. Here, we will store each record in 2 different heaps, a min-heap to efficiently track the lowest stock price and a max-heap to efficiently track the highest stock price.

Now the real challenge will be, how to update stock prices?

We can directly change the stock price in the hashmap, but in the heaps, we would have to pop all stock prices until the old price comes on top and then push the new price and all other popped prices back. This would make the update operation very costly.

One way to resolve this issue is every time we get a new price, we push it into each heap, and only while getting the top element we need to verify if the price is correct or outdated.

But how do we know which prices are outdated?

For this, we can use our hashmap (timestampPriceMap). Every time we receive a (price, timestamp) pair, we will set the value for the key (timestamp) to the given price. If the timestamp already exists in the hashmap, we overwrite the old price. So, when finding the maximum or minimum price, we will check to see if the (price, timestamp) pair on the top of the heap agrees with the price listed for the timestamp in the hashmap. If it does not, then the price is outdated, and we discard this pair and get the next top element and again check with the hashmap.

Insertion/Updation of Record:

    * Insert/Update the stock price at the current timestamp in timestampPriceMap.
    * Push the (price, timestamp) pair into the minHeap and maxHeap.

Get the Latest Price:

    * Use one variable to keep track of the latest time and get the stock's price at the latest time from timestampPriceMap.

Get Minimum and Maximum Stock Price:

    * Get the (price, timestamp) pair from the top of minHeap/maxHeap.
    * If timestampPriceMap[timestamp] != price, it means the price for the current timestamp was updated and that this price is outdated. So we discard this pair and repeat the above step. Otherwise, return the current price.

**Algorithm**

* Initialize variables:
    * latestTime, variable to store the latest timestamp according to the records.
    * timestampPriceMap, a hashmap to store timestamp and prices of the stock.
    * minHeap, maxHeap, heaps to store (price, timestamp) pairs and sort elements based on price.
* In the update function, the current record can be a new record or a correction to an old record:
    * Try to update latestTime, to the current timestamp.
    * Add/update the current timestamp's price in timestampPriceMap.
    * Push (price, timestamp) pair in the heaps.
* In the current function, we need to return the latest price of the stock, i.e. price of the stock at latestTime in timestampPriceMap.
* In the maximum / minimum function, we get the (price, timestamp) pair from the top of maxHeap / minHeap. If timestampPriceMap[timestamp] is not same as price, we discard this pair and repeat the same step again. Otherwise, return the current price.

**Implementation**
```callout
Note: In python, in min-heap we push stock prices after multiplying with -1 so that the min-heap behaves as a max-heap. This helps in keeping the implementation simpler.
```

**Complexity Analysis**

Let NN be the number of records in the input stream.

* Time complexity: O(N \log N)O(NlogN)

    * In the update function, we add one record to the hashmap and to each heap. Adding the record to the hashmap takes constant time. However, for a heap, each push operation takes O(\log N)O(logN) time. So for NN update calls, it will take O(N\log N)O(NlogN) worst-case time.

    * Each current function call takes only constant time to return the result.

    * In the maximum and minimum functions, we pop any outdated records that are at the top of the heap. In the worst-case scenario, we might pop (N - 1)(N−1) elements and each pop takes O(\log N)O(logN) time, so it might seem for one function call the time complexity is N \log NNlogN, so for NN functions calls it could be N^{2} \log NN 
    2
    logN. However, when we pop a record from the heap, it's gone and won't be popped again. So overall, if we push NN elements into a heap, we cannot pop more than NN elements, taking into account all function calls. Thus, calls to maximum and minimum will at most require O(N \log N)O(NlogN) time.

* Space complexity: O(N)O(N)

    * In the update function, we add a record to the hashmap and each heap. Since each stock price takes O(1)O(1) space, for NN update calls, it will take O(N)O(N) space.

    * The current function does not use any additional space.

    * In the maximum and minimum functions, we only remove elements from the heap thus these functions also do not use any additional space.

    * Thus, in the worst-case, we will add all NN records to the hashmap and to both heaps, which takes O(N)O(N) space.



In [51]:
from heapq import *
class StockPrice:
    def __init__(self):
        self.latest_time = 0
        # Store price of each stock at each timestamp.
        self.timestamp_price_map = {}
        
        # Store stock prices in sorted order to get min and max price.
        self.max_heap = []
        self.min_heap = []

    def update(self, timestamp: int, price: int) -> None:
        # Update latestTime to latest timestamp.
        self.timestamp_price_map[timestamp] = price
        self.latest_time = max(self.latest_time, timestamp)

        # Add latest price for timestamp.
        heappush(self.min_heap, (price, timestamp))
        heappush(self.max_heap, (-price, timestamp))

    def current(self) -> int:
        # Return latest price of the stock.
        return self.timestamp_price_map[self.latest_time]

    def maximum(self) -> int:
        price, timestamp = self.max_heap[0]

        # Pop pairs from heap with the price doesn't match with hashmap.
        while -price != self.timestamp_price_map[timestamp]:
            heappop(self.max_heap)
            price, timestamp = self.max_heap[0]
            
        return -price

    def minimum(self) -> int:
        price, timestamp = self.min_heap[0]

        # Pop pairs from heap with the price doesn't match with hashmap.
        while price != self.timestamp_price_map[timestamp]:
            heappop(self.min_heap)
            price, timestamp = self.min_heap[0]
            
        return price

# Driver Code
if __name__ == "__main__":
    stock = StockPrice()
    stock.update(1, 10)
    stock.update(2, 5)
    print(stock.current())
    print(stock.maximum())
    stock.update(1, 3)
    print(stock.maximum())
    stock.update(4, 2)
    print(stock.minimum())

5
10
5
2


# 9. Find All Possible Recipes from Given Supplies

You have information about n different recipes. You are given a string array recipes and a 2D string array ingredients. The ith recipe has the name recipes[i], and you can create it if you have all the needed ingredients from ingredients[i]. Ingredients to a recipe may need to be created from other recipes, i.e., ingredients[i] may contain a string that is in recipes.

You are also given a string array supplies containing all the ingredients that you initially have, and you have an infinite supply of all of them.

Return a list of all the recipes that you can create. You may return the answer in any order.

Note that two recipes may contain each other in their ingredients.


```
Example 1:

Input: recipes = ["bread"], ingredients = [["yeast","flour"]], supplies = ["yeast","flour","corn"]
Output: ["bread"]
Explanation:
We can create "bread" since we have the ingredients "yeast" and "flour".
Example 2:

Input: recipes = ["bread","sandwich"], ingredients = [["yeast","flour"],["bread","meat"]], supplies = ["yeast","flour","meat"]
Output: ["bread","sandwich"]
Explanation:
We can create "bread" since we have the ingredients "yeast" and "flour".
We can create "sandwich" since we have the ingredient "meat" and can create the ingredient "bread".
Example 3:

Input: recipes = ["bread","sandwich","burger"], ingredients = [["yeast","flour"],["bread","meat"],["sandwich","meat","bread"]], supplies = ["yeast","flour","meat"]
Output: ["bread","sandwich","burger"]
Explanation:
We can create "bread" since we have the ingredients "yeast" and "flour".
We can create "sandwich" since we have the ingredient "meat" and can create the ingredient "bread".
We can create "burger" since we have the ingredient "meat" and can create the ingredients "bread" and "sandwich".
```

## Algorithm: DFS

We can use a simple DFS; we just need to track can_make for each recipe (undefined, yes or no), so that we traverse each node only once.

To simplify the DFS logic, we check supplies when generating graph, and create a self-loop for recipes without needed supplies. This ensures that those recipes cannot be made.

1. Create a graph/hashmap to store the recipe and its ingredients that are missing from the supplies.
2. The idea is to start with a recipe and check the availability of all the ingredients that the recipe requires, and if any of the ingredients is missing, then repeat the same to see if that missing ingredient(s) can be found within the recipes that may have been prepared already( achieved by calling dfs recursively ). Here, the node is a recipe, and the dictionary graph contains the mapping between a recipe and the corresponding ingredient that is missing from the supplies array.
The can_make dictionary is used to mark if a recipe can be prepared or not.

```
can_make[recipe] = all([dfs(ingr) for ingr in graph[recipe]]) yields True for a recipe that has all the required ingredients in the supplies array, which roughly translates to
can_make[recipe] = all([]) for a recipe say bread that has all the ingredients ['yeast', 'flour'] in the supplies array, which is nothing but the value True.
```


In [55]:
class Solution:
    def findAllRecipes(self, recipes, ingredients, supplies):
        # if ingredient of recipe exist in supplies, then it's good to go
        # if ingredient of recipe doesn't exist in supplies, then use dfs  to check if we can make it from the other ingredients
        
        supplies = set(supplies)
        graph = {}
        res = []

        for recipe, ingredient in zip(recipes, ingredients):
            graph[recipe] = ingredient
        
        visited = set() # to avoid cycles
        def dfs(recipe):
            # if recipe hasn't been visited but it exists in graph, then we can make it
            # if ingredient of recipe doesn't exist and it's already visited or doesn't come from other recipes (graph), then we return False since we don't have enough ingredients
            if recipe in visited:
                return False
            visited.add(recipe)
            if recipe not in graph:
                return False
            for ingredient in graph[recipe]:
                if ingredient not in supplies:
                    if not dfs(ingredient):
                        return False
            res.append(recipe)
            supplies.add(recipe)
            return True
        
        for recipe in recipes:
            dfs(recipe)
        return res
        

# Driver Code
if __name__ == "__main__":
    recipes = ["bread"]
    ingredients = [["yeast","flour"]]
    supplies = ["yeast","flour","corn"]
    print(Solution().findAllRecipes(recipes, ingredients, supplies))


['bread']


# 10. Maximum Number of Points with Cost

You are given an m x n integer matrix points (0-indexed). Starting with 0 points, you want to maximize the number of points you can get from the matrix.

To gain points, you must pick one cell in each row. Picking the cell at coordinates (r, c) will add points[r][c] to your score.

However, you will lose points if you pick a cell too far from the cell that you picked in the previous row. For every two adjacent rows r and r + 1 (where 0 <= r < m - 1), picking cells at coordinates (r, c1) and (r + 1, c2) will subtract abs(c1 - c2) from your score.

Return the maximum number of points you can achieve.

abs(x) is defined as:

x for x >= 0.
-x for x < 0.
 
```
Example 1:
Input: points = [[1,2,3],[1,5,1],[3,1,1]]
Output: 9
Explanation:
The blue cells denote the optimal cells to pick, which have coordinates (0, 2), (1, 1), and (2, 0).
You add 3 + 5 + 3 = 11 to your score.
However, you must subtract abs(2 - 1) + abs(1 - 0) = 2 from your score.
Your final score is 11 - 2 = 9.
```

O(M*N) time and O(N) space


In [57]:
from collections import deque


class Solution:
    def maxPoints(self, points):
        """ DP in Time: O(M*N) Space: O(N)
        
        Define dp[i][j] (i = 0, ..., m - 1; j = 0, ..., n - 1) as max points that one can get from 
        the first i rows (up to the i-th row), and for the i-th row, picking exactly the j-th point
        
        fill dp row by row until the (m - 1)-th row, then return max(dp[-1]) as max points 
        we have initial case: first row dp[0][j] = points[0][j] for j in range(n)
        and for i >= 1: dp[i][j] = points[i][j] + max(dp[i-1][k] - abs(k-j) for k in range(n))
               
        focus on the max term: max(dp[i-1][k] - abs(k-j) for k in range(n)) = max(A[k] + B[k] for k in range(n))
        where A = dp[i-1] remains unchanged over different j, only B shifts right with j
        For example: 
        for each j=0,...,n-1, we need to find the max over A[k] + B[k] to fill dp[i][j]
        A: ..., dp[i-1][j-2], dp[i-1][j-1], dp[i-1][j], dp[i-1][j+1], dp[i-1][j+2], ...
        B: ...,           -2,           -1,          0,           -1,           -2, ...
        
        for j = 0, ..., n-1, 
        there are repetative patterns in this max(A+B) calculation that can be speed up using another DP
        
        for each j
        define left[j] (j=0, ..., n-1) as left[j] = max(A[:j+1] + B[:j+1]) 
        left[0] = max(dp[i-1][0])
        left[1] = max(dp[i-1][0] - 1, dp[i-1][1]) = max(left[0] - 1, dp[i-1][1])
        left[2] = max(dp[i-1][0] - 2, dp[i-1][1] - 1, dp[i-1][2]) = max(left[1] - 1, dp[i-1][2])
        left[3] = max(dp[i-1][0] - 3, dp[i-1][1] - 2, dp[i-1][2] - 1, dp[i-1][3]) = max(left[2] - 1, dp[i-1][3])
        ...
        left can be filled in linear time 
        
        for each j define right[j] (j=0, ..., n-1) as right[j] = max(A[j:] + B[j:])
        right[n-1] = max(dp[i-1][n-1])
        right[n-2] = max(dp[i-1][n-2], dp[i-1][n-1] - 1) = max(dp[i-1][n-2], right[n-1]-1)
        right[n-3] = max(dp[i-1][n-3], dp[i-1][n-2] - 1 , dp[i-1][n-1] - 2) = max(dp[i-1][n-3], right[n-2] - 1)
        ...
        right[0] = max(dp[i-1][0], right[1] - 1)
        right can be filled in linear time
        
        max(left[j], right[j]) 
        = max( 
                max(A[:j+1] + B[:j+1]), 
                max(A[j:]   + B[j:])        
            ) = max(A + B) = max(dp[i-1][k] - abs(k-j) for k in range(n))
            
        thus dp[i][j] = points[i][j] + max(dp[i-1][k] - abs(k-j) for k in range(n)) 
         = points[i][j] + max(left[j], right[j]) 
         can be computed in O(1) time
        """
        num_rows, num_cols = len(points), len(points[0])
        dp_cur, dp_prev = points[0], points[0]
        for i in range(1, num_rows):
            # construct left
            for j in range(num_cols):
                if j == 0:
                    left = [dp_prev[0]]
                else:
                    left.append(max(left[-1] - 1, dp_prev[j]))
            # construct right
            for j in reversed(range(num_cols)):
                if j == num_cols - 1:
                    right = deque([dp_prev[j]])
                else:
                    right.appendleft(max(dp_prev[j], right[0] - 1))
            # fill dp[i][j]
            for j in range(num_cols):
                dp_cur[j] = points[i][j] + max(left[j], right[j])
            dp_prev = dp_cur.copy()
        return max(dp_cur)

# Driver Code
if __name__ == "__main__":
    points = [[1,2,3],[1,5,1],[3,1,1]]
    print(Solution().maxPoints(points))
        


9


# 11. Amount of New Area Painted Each Day

There is a long and thin painting that can be represented by a number line. You are given a 0-indexed 2D integer array paint of length n, where paint[i] = [starti, endi]. This means that on the ith day you need to paint the area between starti and endi.

Painting the same area multiple times will create an uneven painting so you only want to paint each area of the painting at most once.

Return an integer array worklog of length n, where worklog[i] is the amount of new area that you painted on the ith day.

Example 1:
![img](https://assets.leetcode.com/uploads/2022/02/01/screenshot-2022-02-01-at-17-16-16-diagram-drawio-diagrams-net.png)
```
Input: paint = [[1,4],[4,7],[5,8]]
Output: [3,3,1]
Explanation:
On day 0, paint everything between 1 and 4.
The amount of new area painted on day 0 is 4 - 1 = 3.
On day 1, paint everything between 4 and 7.
The amount of new area painted on day 1 is 7 - 4 = 3.
On day 2, paint everything between 7 and 8.
Everything between 5 and 7 was already painted on day 1.
The amount of new area painted on day 2 is 8 - 7 = 1. 
```
Example 2:
![img](https://assets.leetcode.com/uploads/2022/02/01/screenshot-2022-02-01-at-17-17-45-diagram-drawio-diagrams-net.png)

```
Input: paint = [[1,4],[5,8],[4,7]]
Output: [3,3,1]
Explanation:
On day 0, paint everything between 1 and 4.
The amount of new area painted on day 0 is 4 - 1 = 3.
On day 1, paint everything between 5 and 8.
The amount of new area painted on day 1 is 8 - 5 = 3.
On day 2, paint everything between 4 and 5.
Everything between 5 and 7 was already painted on day 1.
The amount of new area painted on day 2 is 5 - 4 = 1. 
```
Example 3:
![img](https://assets.leetcode.com/uploads/2022/02/01/screenshot-2022-02-01-at-17-19-49-diagram-drawio-diagrams-net.png)
```
Input: paint = [[1,5],[2,4]]
Output: [4,0]
Explanation:
On day 0, paint everything between 1 and 5.
The amount of new area painted on day 0 is 5 - 1 = 4.
On day 1, paint nothing because everything between 2 and 4 was already painted on day 0.
The amount of new area painted on day 1 is 0.
```

This is a question testing your data structure skill, the more advanced data structure you know, the easier it will be. I will provide 3 different solutions

- Red-Black Tree (sortedcontainer in python & set in c++)
- Prioirty Queue & Set
- Segment Tree

The basic concept is called sweep line.

For example paint = [[1,4],[5,8],[4,7]], The picture would be like below

![img](https://assets.leetcode.com/users/images/6247c212-6c87-4314-b3e8-f21fee39e3fd_1643862042.2531443.png)

First, put each [start, end] as brown lines, put them on top of the number line one by one ordered by index. and imagine a blue line which will sweep from left to right across all the brown lines.

Moveover, we need another box which can:

- add number
- delete choosen number
- get minimum number

we will discuss the suitable data structure for this box later.

Now, when the blue line sweeps, every time when it touches a start position, it will add the index number to the box, and when it touches an end position, it will remove the index number from the box as the picture shows below.

![img](https://assets.leetcode.com/users/images/e2320b81-40de-4b95-ae06-a57637d4addb_1643998733.061133.png)

Now, the answer is very easy, in each position, you will look at the box and get the minimum number from it. If it has a minimum number, it's the index you should +1 for the answer. Since it means this is the index show up earliest at this position.

## Approach: Using Priority Queue

- The priority queue could add and get minimum
- The set will be used to record those indexes which already ended.

Each time when you want to get minimum number from the priority queue, you need to discard all those ended indexes already recorded in the set.

# Algorithm
1. Go through each index in paint, and add its index to the heap.
2. 


In [61]:
class Solution:
    def amountPainted(self, paint):
        # constructure the sweep line
        records = []
        max_pos = 0
        for i, [start, end] in enumerate(paint):
            records.append((start, i, 1)) # use 1 and -1 to records the type.
            records.append((end, i, -1))
            max_pos = max(max_pos, end)
        records.sort()
        print(f"records is {records}")

        # sweep across all position
        ans = [0 for _ in range(len(paint))]
        indexes = []
        ended_set = set()
        i = 0
        for pos in range(max_pos + 1):
            print(f"Initial Pos is {pos}")
            while i < len(records) and records[i][0] == pos:
                pos, index, type = records[i]
                if type == 1:
                    heapq.heappush(indexes, index)
                else:
                    ended_set.add(index)
                i += 1
            print(f"Indexes after adding to heapq: {indexes}")
            print(f"Ended set: {ended_set}")
            while indexes and indexes[0] in ended_set:
                heapq.heappop(indexes)

            if indexes:
                ans[indexes[0]] += 1
        return ans
# Driver Code
if __name__ == "__main__":
    paint = [[1,4],[5,8],[4,7]]
    print(Solution().amountPainted(paint))

records is [(1, 0, 1), (4, 0, -1), (4, 2, 1), (5, 1, 1), (7, 2, -1), (8, 1, -1)]
Initial Pos is 0
Indexes after adding to heapq: []
Ended set: set()
Initial Pos is 1
Indexes after adding to heapq: [0]
Ended set: set()
Initial Pos is 2
Indexes after adding to heapq: [0]
Ended set: set()
Initial Pos is 3
Indexes after adding to heapq: [0]
Ended set: set()
Initial Pos is 4
Indexes after adding to heapq: [0, 2]
Ended set: {0}
Initial Pos is 5
Indexes after adding to heapq: [1, 2]
Ended set: {0}
Initial Pos is 6
Indexes after adding to heapq: [1, 2]
Ended set: {0}
Initial Pos is 7
Indexes after adding to heapq: [1, 2]
Ended set: {0, 2}
Initial Pos is 8
Indexes after adding to heapq: [1, 2]
Ended set: {0, 1, 2}
[3, 3, 1]


# 12. Snapshot Array

Implement a SnapshotArray that supports the following interface:

* SnapshotArray(int length) initializes an array-like data structure with the given length.  Initially, each element equals 0.
* void set(index, val) sets the element at the given index to be equal to val.
* int snap() takes a snapshot of the array and returns the snap_id: the total number of times we called snap() minus 1.
* int get(index, snap_id) returns the value at the given index, at the time we took the snapshot with the given snap_id
 
```
Example 1:

Input: ["SnapshotArray","set","snap","set","get"]
[[3],[0,5],[],[0,6],[0,0]]
Output: [null,null,0,null,5]
Explanation: 
SnapshotArray snapshotArr = new SnapshotArray(3); // set the length to be 3
snapshotArr.set(0,5);  // Set array[0] = 5
snapshotArr.snap();  // Take a snapshot, return snap_id = 0
snapshotArr.set(0,6);
snapshotArr.get(0,0);  // Get the value of array[0] with snap_id = 0, return 5
```
```
Intuition
Instead of copy the whole array,
we can only record the changes of set.


Explanation
The idea is, the whole array can be large,
and we may take the snap tons of times.
(Like you may always ctrl + S twice)

Instead of record the history of the whole array,
we will record the history of each cell.
And this is the minimum space that we need to record all information.

For each A[i], we will record its history.
With a snap_id and a its value.

When we want to get the value in history, just binary search the time point.


Complexity
Time O(logS)
Space O(S)
where S is the number of set called.

SnapshotArray(int length) is O(N) time
set(int index, int val) is O(1) in Python and O(logSnap) in Java
snap() is O(1)
get(int index, int snap_id) is O(logSnap)
```

In [73]:
import bisect
class SnapshotArray(object):
    
    def __init__(self, length):
        self.d = collections.defaultdict(list)
        self.snap_id = 0

    def set(self, index, val):
        self.d[index].append((self.snap_id, val))
        print(f"self.d is {self.d}")

    def snap(self):
        self.snap_id += 1
        return self.snap_id - 1

    def get(self, index, snap_id):
        i = bisect.bisect(self.d[index], (snap_id, float('inf'))) - 1
        print(f"i is {i}")
        if index in self.d and i != -1:
            return self.d[index][i][1]
        else:
            return 0

# Driver Code
if __name__ == "__main__":
    s = SnapshotArray(3)
    s.set(0, 5)
    snap_id = s.snap()
    print(f"snap_id is {snap_id}")
    s.set(0, 6)
    print(s.get(0, snap_id))


self.d is defaultdict(<class 'list'>, {0: [(0, 5)]})
snap_id is 0
self.d is defaultdict(<class 'list'>, {0: [(0, 5), (1, 6)]})
i is 0
5


# 13. Maximum Split of Positive Even Integers

You are given an integer finalSum. Split it into a sum of a maximum number of unique positive even integers.

For example, given finalSum = 12, the following splits are valid (unique positive even integers summing up to finalSum): (12), (2 + 10), (2 + 4 + 6), and (4 + 8). Among them, (2 + 4 + 6) contains the maximum number of integers. Note that finalSum cannot be split into (2 + 2 + 4 + 4) as all the numbers should be unique.
Return a list of integers that represent a valid split containing a maximum number of integers. If no valid split exists for finalSum, return an empty list. You may return the integers in any order.

 
```
Example 1:

Input: finalSum = 12
Output: [2,4,6]
Explanation: The following are valid splits: (12), (2 + 10), (2 + 4 + 6), and (4 + 8).
(2 + 4 + 6) has the maximum number of integers, which is 3. Thus, we return [2,4,6].
Note that [2,6,4], [6,2,4], etc. are also accepted.
Example 2:

Input: finalSum = 7
Output: []
Explanation: There are no valid splits for the given finalSum.
Thus, we return an empty array.
Example 3:

Input: finalSum = 28
Output: [6,8,2,12]
Explanation: The following are valid splits: (2 + 26), (6 + 8 + 2 + 12), and (4 + 24). 
(6 + 8 + 2 + 12) has the maximum number of integers, which is 4. Thus, we return [6,8,2,12].
Note that [10,2,4,12], [6,2,4,16], etc. are also accepted.
```

## Approach:
```
So it's clear from question that if n is odd answer is not possible (bcz we can't represent a odd number as a sum of even numbers)

Now if n is even , then we have to make the largest list of unique even number such that the sum is equal to given n.

Now to make the largest we have to take smallest numbers first like 2,4,6,8... and so on.

but wait what happen if we are doing in this manner and the total sum is greater than desired , no worries , we wll do this step untill our sum is less than or equal to given number , and just add the remaining difference to last number in the list.

Take n =14

i = 2 , crSum = 0 , list = [] (crSum + 2 <= 14 , so push it) , crSum + i = 2 , list = [2]
i = 4 , crSum = 2 , list = [2] (crSum + 4 <= 14 , so push it) , crSum + i = 6 , list = [2,4]
i = 6 , crSum = 6 , list = [2,4] (crSum + 6 <= 14 , so push it) , crSum + i = 12 , list = [2,4,6]
i = 8 , crSum = 12 , list = [2,4,6] (crSum + 8 > 14 , so don't push it , break the loop)
Now we have crSum = 12 , and we want 14 , so simply add difference (which is 14-12 = 2 ) in the last element of list

so list = [2,4,6+(14-12)] = [2,4,8]
```

In [85]:
class Solution:
    def maximumEvenSplit(self, finalSum):
        if finalSum % 2 != 0:
            return 0
        num = 2
        temp_sum = 0
        list_sum = []
        while temp_sum + num <= finalSum:
            list_sum.append(num)
            temp_sum += num
            num += 2
            
            print(f"temp_sum is {temp_sum}")
        # add the difference between finalSum and temp_sum to the last element of list_sum
        
        list_sum[-1] += finalSum - temp_sum
        return list_sum 

# Driver Code
if __name__ == "__main__":
    print(Solution().maximumEvenSplit(10))

temp_sum is 2
temp_sum is 6
[2, 8]


# 14. Logger Rate Limiter

Design a logger system that receives a stream of messages along with their timestamps. Each unique message should only be printed at most every 10 seconds (i.e. a message printed at timestamp t will prevent other identical messages from being printed until timestamp t + 10).

All messages will come in chronological order. Several messages may arrive at the same timestamp.

Implement the Logger class:

Logger() Initializes the logger object.
bool shouldPrintMessage(int timestamp, string message) Returns true if the message should be printed in the given timestamp, otherwise returns false.
 
```
Example 1:

Input
["Logger", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage", "shouldPrintMessage"]
[[], [1, "foo"], [2, "bar"], [3, "foo"], [8, "bar"], [10, "foo"], [11, "foo"]]
Output
[null, true, true, false, false, false, true]

Explanation
Logger logger = new Logger();
logger.shouldPrintMessage(1, "foo");  // return true, next allowed timestamp for "foo" is 1 + 10 = 11
logger.shouldPrintMessage(2, "bar");  // return true, next allowed timestamp for "bar" is 2 + 10 = 12
logger.shouldPrintMessage(3, "foo");  // 3 < 11, return false
logger.shouldPrintMessage(8, "bar");  // 8 < 12, return false
logger.shouldPrintMessage(10, "foo"); // 10 < 11, return false
logger.shouldPrintMessage(11, "foo"); // 11 >= 11, return true, next allowed timestamp for "foo" is 11 + 10 = 21
```

Approach : Hashtable / Dictionary

Intuition

One could combine the queue and set data structure into a hashtable or dictionary, which gives us the capacity of keeping all unique messages as of queue as well as the capacity to quickly evaluate the duplication of messages as of set.

The idea is that we keep a hashtable/dictionary with the message as key, and its timestamp as the value. The hashtable keeps all the unique messages along with the latest timestamp that the message was printed.
![img](https://leetcode.com/problems/logger-rate-limiter/Figures/359/359_hashtable.png)

As one can see from the above example, there is an entry in the hashtable with the message m2 and the timestamp 2. Then there comes another message m2 with the timestamp 15. Since the message was printed 13 seconds before (i.e. beyond the buffer window), it is therefore eligible to print again the message. As a result, the timestamp of the message m2 would be updated to 15.

Algorithm

* We initialize a hashtable/dictionary to keep the messages along with the timestamp.

* At the arrival of a new message, the message is eligible to be printed with either of the two conditions as follows:

    * case 1). we have never seen the message before.

    * case 2). we have seen the message before, and it was printed more than 10 seconds ago.

* In both of the above cases, we would then update the entry that is associated with the message in the hashtable, with the latest timestamp.

Complexity Analysis

Time Complexity: \mathcal{O}(1)O(1). The lookup and update of the hashtable takes a constant time.

Space Complexity: \mathcal{O}(M)O(M) where MM is the size of all incoming messages. Over the time, the hashtable would have an entry for each unique message that has appeared.


In [86]:
class Logger:
    
    def __init__(self):
        self.dictionary = {}
        

    def shouldPrintMessage(self, timestamp: int, message: str) -> bool:
        if message not in self.dictionary or self.dictionary[message] + 10 <= timestamp:
            self.dictionary[message] = timestamp
            return True
        else:
            return False

# Driver Code
if __name__ == "__main__":
    l = Logger()
    print(l.shouldPrintMessage(1,"foo"))
    print(l.shouldPrintMessage(2,"bar"))
    print(l.shouldPrintMessage(3,"foo"))
    print(l.shouldPrintMessage(8,"bar"))
    print(l.shouldPrintMessage(10,"foo"))
    print(l.shouldPrintMessage(11,"foo"))


True
True
False
False
False
True


# 15. Detect Squares

You are given a stream of points on the X-Y plane. Design an algorithm that:

Adds new points from the stream into a data structure. Duplicate points are allowed and should be treated as different points.
Given a query point, counts the number of ways to choose three points from the data structure such that the three points and the query point form an axis-aligned square with positive area.
An axis-aligned square is a square whose edges are all the same length and are either parallel or perpendicular to the x-axis and y-axis.

Implement the DetectSquares class:

* DetectSquares() Initializes the object with an empty data structure.
* void add(int[] point) Adds a new point point = [x, y] to the data structure.
* int count(int[] point) Counts the number of ways to form axis-aligned squares with point point = [x, y] as described above.

Example 1:

![img](https://assets.leetcode.com/uploads/2021/09/01/image.png)
```
Input
["DetectSquares", "add", "add", "add", "count", "count", "add", "count"]
[[], [[3, 10]], [[11, 2]], [[3, 2]], [[11, 10]], [[14, 8]], [[11, 2]], [[11, 10]]]
Output
[null, null, null, null, 1, 0, null, 2]

Explanation
DetectSquares detectSquares = new DetectSquares();
detectSquares.add([3, 10]);
detectSquares.add([11, 2]);
detectSquares.add([3, 2]);
detectSquares.count([11, 10]); // return 1. You can choose:
                               //   - The first, second, and third points
detectSquares.count([14, 8]);  // return 0. The query point cannot form a square with any points in the data structure.
detectSquares.add([11, 2]);    // Adding duplicate points is allowed.
detectSquares.count([11, 10]); // return 2. You can choose:
                               //   - The first, second, and third points
                               //   - The first, third, and fourth points
```

Solution : Given p1, try all points p3 (p1 and p3 form diagonal)
```
To compute count(p1):
We try all points p3 which together with p1 form the diagonal of non-empty square, it means abs(p1.x-p3.x) == abs(p1.y-p3.y) && abs(p1.x-p3.x) > 0
Since we have 2 points p1 and p3, we can form a square by computing the positions of 2 remain points p2, p4.
p2 = (p1.x, p3.y)
p4 = (p3.x, p1.y)
```

![img](https://assets.leetcode.com/users/images/3f33581d-baa5-4fd4-9516-a1098af539d1_1632034530.1139376.png)


Complexity

Time:
add: O(1)
count: O(T), where T <= 5000 is total number of points after calling add.
Space: O(T)


In [91]:
from collections import Counter
class DetectSquares:
    def __init__(self):
        self.cntPoints = Counter()

    def add(self, point) -> None:
        self.cntPoints[tuple(point)] += 1

    def count(self, point) -> int:
        ans = 0
        x1, y1 = point
        for (x3, y3), cnt in self.cntPoints.items():
            if abs(x1 - x3) == 0 or abs(x1 - x3) != abs(y1 - y3):
                continue  # Skip empty square or invalid square point!
            ans += cnt * self.cntPoints[(x1, y3)] * self.cntPoints[(x3, y1)]
        return ans

# Driver Code
if __name__ == "__main__":
    d = DetectSquares()
    d.add([3, 10])
    d.add([11, 2])
    d.add([3, 2])
    print(d.count([11, 10]))

1


# 16. Maximum Number of Visible Points

You are given an array points, an integer angle, and your location, where location = [posx, posy] and points[i] = [xi, yi] both denote integral coordinates on the X-Y plane.

Initially, you are facing directly east from your position. You cannot move from your position, but you can rotate. In other words, posx and posy cannot be changed. Your field of view in degrees is represented by angle, determining how wide you can see from any given view direction. Let d be the amount in degrees that you rotate counterclockwise. Then, your field of view is the inclusive range of angles [d - angle/2, d + angle/2].

You can see some set of points if, for each point, the angle formed by the point, your position, and the immediate east direction from your position is in your field of view.

There can be multiple points at one coordinate. There may be points at your location, and you can always see these points regardless of your rotation. Points do not obstruct your vision to other points.

Return the maximum number of points you can see.

Example 1:
![img](https://assets.leetcode.com/uploads/2020/09/30/89a07e9b-00ab-4967-976a-c723b2aa8656.png)

```
Input: points = [[2,1],[2,2],[3,3]], angle = 90, location = [1,1]
Output: 3
Explanation: The shaded region represents your field of view. All points can be made visible in your field of view, including [3,3] even though [2,2] is in front and in the same line of sight.
```

Example 2:
```
Input: points = [[2,1],[2,2],[3,4],[1,1]], angle = 90, location = [1,1]
Output: 4
Explanation: All points can be made visible in your field of view, including the one at your location.
```

Example 3:
![img](https://assets.leetcode.com/uploads/2020/09/30/5010bfd3-86e6-465f-ac64-e9df941d2e49.png)

```
Input: points = [[1,0],[2,1]], angle = 13, location = [1,1]
Output: 1
Explanation: You can only see one of the two points, as shown above.
```

Solution Explanation:

First, we find all the angles that the points make with respect to our central point. Since this is atan2, this is measured counterclockwise from the positive x-axis. We store all these angles for our next computation. For duplicate central points, we only store their counts and do not calculate angles for them. As mentioned in the question, these are part of the view, so we will just have to return it as an addition to our answer. Imagine the central point is now origin and axes pass through it.

![img](https://assets.leetcode.com/users/images/e185ff6e-5ff9-4829-993f-a016ff28bb76_1601806825.3893652.png)


Sort this angles array. Angles with points closer to x-axis towards the right of the central point will come first. Basically, arrangement is counterclockwise as discussed above.
If you draw all this on a sheet of paper and extend all lines from the centre to each point to infinity, you will notice that all angles are basically sectors of an imaginary circle. Now, we need to start with a line and see how many portions of these irregular pizza slices we can include such that the total angle is less than or equal to given target angle. We will start from the first angle of our array that represents the first line above positive x-axis (wrt central point). Now all we have to do is go around this circle and calculate differences between these array values. Each angle value is wrt +ve x-axis, so the difference represents angle formed between two points with central point, which is what we need.

What is important is the +360 step. This is because our array is no longer linear, it's circular. So the first element could be a part of the calculation we are performing towards the end of the array. Basically meaning points below the +ve-axis, now going counterclockwise, can include array elements at the beginning. So we just add extra data where each angle is extended by 360. So we go around the circle twice and include all cases.

(Imagine we are coming around, the first circular path has completed in our calculation. When we run into the elements at the start of the array again, they will have to be greater since we are subtracting the prev value from the next. Let's say last angle in the first round was 300 degrees and we are back to 30 degrees at the start. But 30-300 won't make any sense. That 30 is basically a full circle +30, so it should be 360+30=390, and then 390-300 = 90 degrees difference, which is how it intuitively looks when you see the graph. That's it).

![img](https://assets.leetcode.com/users/images/14e19c19-bcc7-44d4-8cab-3bc619601bd9_1601807292.3661292.png)

If there is any confusion regarding the final loop, I will explain that too. Let's say we start with the first angle and move counter-clockwise. The difference is within limits, so we add it to our counter. How? Difference of indexes + 1. We start with j=0 and i goes on incrementing as it satisfies the condition, so the indexes are basically the points. First, second, third, all the points satisfy difference with first value within target angle. Now comes an angle where difference with our very first angle is greater than required. So obviously, we need to move forward. A naive way of this would be to break, assuming we are done with the first angle and now we will start with the second angle (Second angle wrt third angle, second angle wrt fourth angle, second angle wrt fifth angle and so on, basically your quadratic O(n^2)). But if you see closely, these are duplicate calculations. If the difference between first and third angle was within limits, so will be the difference between second and third angles, because it is obviously lesser. So, if the difference between fourth angle and first angle is not satisfied, we can check the difference of fourth angle and the second angle. If it is within limits, we add a counter and check if it is better than our maximum, using the same logic as above for the first angle. And so on and so forth we go, twice around this circle, finding differences, calculating number of points and comparing it with the max. At the end of this, we have our final answer. Don't forget to add the extra duplicate points at the end!

So all in all, this is a circular sliding window solution to this problem.

Idea

Here are the steps:
```
convert all coordinates to radians
sort the array
use sliding window to find the longest window that satisfies arr[r] - arr[l] <= angle.
Note that we need to go around the circle, so we duplicate the array and offset the second half by 2*pi.
```

Complexity

Time complexity: O(NlogN)

Space complexity: O(N)


In [95]:
import math
class Solution:
    def visiblePoints(self, points, angle, location):
        # Duplicate points 
        duplicate_points = 0
        # Array that contains the angle in radian formed by the point and the location w/ x-axis
        angles = []

        xx, yy = location # x, y coordinates of the location

        # go through all the points
        for x, y in points:
            # if duplicate point, increment duplicate_points
            if x == xx and y == yy:
                duplicate_points += 1
                continue
            # calculate the angle formed by the point and the location
            angles.append(math.atan2(y - yy, x - xx))
        
        # sort the angles so that we get the I quadrant first, II quadrant second, III quadrant third, IV quadrant fourth angles in this order
        angles.sort()

        # add 2pi 
        angles = angles + [x + 2 * math.pi for x in angles]
        angle = angle * math.pi / 180

        # start with the leftmost angle of the sliding window
        left = 0
        ans = 0
        # go through all the angles
        for right in range(len(angles)):
            # if the angle formed by right and left is more than the angle we want, then we can move the left pointer
            while angles[right] - angles[left] > angle:
                left += 1
            ans = max(ans, right - left + 1)
        return ans + duplicate_points

# Driver Code
if __name__ == "__main__":
    points = [[2,1],[2,2],[3,3]]
    angle = 90
    location = [1,1]
    print(Solution().visiblePoints(points, angle, location))
            

        



3


# 17. Longest String Chain

You are given an array of words where each word consists of lowercase English letters.

wordA is a predecessor of wordB if and only if we can insert exactly one letter anywhere in wordA without changing the order of the other characters to make it equal to wordB.

For example, "abc" is a predecessor of "abac", while "cba" is not a predecessor of "bcad".
A word chain is a sequence of words [word1, word2, ..., wordk] with k >= 1, where word1 is a predecessor of word2, word2 is a predecessor of word3, and so on. A single word is trivially a word chain with k == 1.

Return the length of the longest possible word chain with words chosen from the given list of words.
```
Example 1:

Input: words = ["a","b","ba","bca","bda","bdca"]
Output: 4
Explanation: One of the longest word chains is ["a","ba","bda","bdca"].
Example 2:

Input: words = ["xbc","pcxbcf","xb","cxbc","pcxbc"]
Output: 5
Explanation: All the words can be put in a word chain ["xb", "xbc", "cxbc", "pcxbc", "pcxbcf"].
Example 3:

Input: words = ["abcd","dbqca"]
Output: 1
Explanation: The trivial word chain ["abcd"] is one of the longest word chains.
["abcd","dbqca"] is not a valid word chain because the ordering of the letters is changed.
```

Overview

A word chain is a sequence of words (word1 -> word2 -> word3 -> word4 -> word5......) such that word1 is a predecessor of word2 and so on. A key point in the problem statement is that word1 can be a predecessor of word2 if and only if we can add exactly one letter anywhere in word1 to make it equal to word2. In other words, word2 should have one letter more than word1 and the position of this new letter can be anywhere. Note that the order of the words in the list does not need to be maintained while creating the word sequence.

Suppose that word1 is ab then word2 can be ab*, a*b, *ab where * is any lowercase English letter.

Therefore, it is possible for a particular word to have more than one predecessor in the given list, and thus belong to more than one word sequence. Our objective is to determine the length of the longest possible word sequence.

Let us consider the following example : ['abcd','abc','bcd','abd','ab','ad','b'] In this list, the immediate predecessors of abcd are ['abc','bcd','abd'] as all these words are missing exactly one letter from the word abcd. Similarly, the immediate predecessors of abd are ['ab','ad'] and the predecessor of ab is ['b'].


Approach : Top-Down Dynamic Programming (Recursion + Memoization)

Intuition

If you're not familiar with DFS (Depth First Search), check out our Explore Card.

Here we work backwards to find the longest chain, this means that we will start from a word and delete one character at a time. We continue this chain until we come across a word that is not present in the list or is one letter long.

In the above example some of the possible word sequences are: abcd -> abd -> ab -> b , abcd -> abc -> ab -> b , abcd -> bcd and so on. The possible word sequences are illustrated in Figure 1.
![img](https://leetcode.com/problems/longest-string-chain/Figures/1048/1048_Overview_Diagram.png)

Figure 1. Figure demonstrating DFS to find the longest word sequence.

In this graph, we can observe that the length of the longest possible word sequence is 4. There are two word sequences that have the longest length : abcd -> abd -> ab -> b and abcd -> abc -> ab -> b. (The longest path is shown in the diagram with red arrows).

Notice that a particular sequence can be a part of more than one word sequence. For example the sequence ab -> b is part of both the following sequences : abcd -> abd -> ab -> b and abcd -> abc -> ab -> b. This leads to repeated calculations because every time we encounter ab we need to explore the subpath ab -> a. For a small list, this is not a problem but as the size of the list increases, the size of the graph grows exponentially.

What we can do is whenever we encounter a new word, we will find all possible sequences with this word as the last word in the sequence. Then, we will store the length of the longest possible sequence that ends with this word.

We will use a map for this where each key will be an ending word and the value will be the length of the longest possible word sequence ending with this word. In the above example when we first encounter the word ab we will store the value 2 (word sequence ab -> b) for key ab. The next time we encounter ab, we will simply return the value stored against it in the map instead of going through the entire subtree again. This process is known as memoization and it prevents recalculation. For every word present in the list, we only need to determine the length of the longest path that ends with this word once.

Algorithm

* Initialize a set (wordsPresent) and add all the words in the list to the set. This set will be used to check if a word is present in the list.

* Initialize a map (memo) having key type as String and value type as Integer. This map will store the length of the longest possible word sequence where the key is the last word in the sequence.

* Iterate over the list. For each word in the list perform a depth-first search.

* In the DFS, consider the current word (currentWord) as the last word in the word sequence.

* If currentWord was encountered previously we just return its corresponding value in the map memo.

* Initialize maxLength to 1.

* Iterate over the entire length of the currentWord.

    * Create all possible words (newWord) by taking out one character at a time.
    * If newWord is present in the set perform a DFS with this word and store the intermediate result in a variable currentLength.
    * Update the maxLength so that it contains the length of the longest sequence possible where the currentWord is the end word.
* Set the maxLength as the value for currentWord (key) in the map.

* Return maxLength.


Complexity Analysis

Let NN be the number of words in the list and LL be the maximum possible length of a word.

Time complexity: O(L ^ 2 \cdot N)O(L 
2
 ⋅N).

Initially, we iterate over the list to store all the given words in a set (adds NN to the complexity).

Next, we perform a DFS for each word (O(N)O(N)). For each word, we iterate over its length(O(L)O(L)). At each index (i) we create a new word by deleting the character at position i from the original word (O(L)O(L)). Therefore, the overall time complexity is O(N + (L ^ 2 \cdot N))O(N+(L 
2
 ⋅N)) = O(L ^ 2 \cdot N))O(L 
2
 ⋅N)), because the NN term is insignificant relative to the L ^ 2 \cdot NL 
2
 ⋅N term. Note that because of memoization we can be sure that each word in the list is traversed only once.

Space complexity: O(N)O(N).

The extra space is used by the recursion call stack. In worst case all the words are a part of the longest word sequence which requires a recursion stack size of NN.

Also, we use a set to store all distinct words (size NN) and a map to store intermediate results (size NN). Since the maximum number of distinct words will be NN (when there is no repetition) the overall space complexity is O(2 \cdot N)O(2⋅N) which in Big O notation equals O(N)O(N).




In [96]:
class Solution:
    def longestStrChain(self, words):
        # set to add all the words in the list
        wordsPresent = set(words)
        # dictionary to store the length of the longest chain for each word, as if the word is the last word in the chain
        longestChain = {}

        ans = 0
        
        for word in words:
            ans = max(ans, self.dfs(wordsPresent, longestChain, word))
        return ans
    
    def dfs(self, wordsPresent, longestChain, word):
        if word in longestChain:
            return longestChain[word]
        maxLength = 1
        for i in range(len(word)):
            # if the word is not in the set, then we can't make a chain from it
            if word[:i] + word[i+1:] not in wordsPresent:
                continue
            # if the word is in the set, then we can make a chain from it
            maxLength = max(maxLength, 1 + self.dfs(wordsPresent, longestChain, word[:i] + word[i+1:]))
        longestChain[word] = maxLength
        return maxLength

# Driver Code
if __name__ == "__main__":
    words = ["a","b","ba","bca","bda","bdca"]
    print(Solution().longestStrChain(words))


4


# 18. Random Pick with Weight

You are given a 0-indexed array of positive integers w where w[i] describes the weight of the ith index.

You need to implement the function pickIndex(), which randomly picks an index in the range [0, w.length - 1] (inclusive) and returns it. The probability of picking an index i is w[i] / sum(w).

For example, if w = [1, 3], the probability of picking index 0 is 1 / (1 + 3) = 0.25 (i.e., 25%), and the probability of picking index 1 is 3 / (1 + 3) = 0.75 (i.e., 75%).
 
```
Example 1:

Input
["Solution","pickIndex"]
[[[1]],[]]
Output
[null,0]

Explanation
Solution solution = new Solution([1]);
solution.pickIndex(); // return 0. The only option is to return 0 since there is only one element in w.
Example 2:

Input
["Solution","pickIndex","pickIndex","pickIndex","pickIndex","pickIndex"]
[[[1,3]],[],[],[],[],[]]
Output
[null,1,1,1,1,0]

Explanation
Solution solution = new Solution([1, 3]);
solution.pickIndex(); // return 1. It is returning the second element (index = 1) that has a probability of 3/4.
solution.pickIndex(); // return 1
solution.pickIndex(); // return 1
solution.pickIndex(); // return 1
solution.pickIndex(); // return 0. It is returning the first element (index = 0) that has a probability of 1/4.

Since this is a randomization problem, multiple answers are allowed.
All of the following outputs can be considered correct:
[null,1,1,1,1,0]
[null,1,1,1,1,1]
[null,1,1,1,0,0]
[null,1,1,1,0,1]
[null,1,0,1,0,0]
......
and so on.
```

Overview

This is actually a very practical problem which appears often in the scenario where we need to do sampling over a set of data.

Nowadays, people talk a lot about machine learning algorithms. As many would reckon, one of the basic operations involved in training a machine learning algorithm (e.g. Decision Tree) is to sample a batch of data and feed them into the model, rather than taking the entire data set. There are several rationales behind doing sampling over data, which we will not cover in detail, since it is not the focus of this article.

If one is interested, one can refer to our Explore card of Machine Learning 101 which gives an overview on the fundamental concepts of machine learning, as well as the Explore card of Decision Tree which explains in detail on how to construct a decision tree algorithm.

Now, given the above background, hopefully one is convinced that this is an interesting problem, and it is definitely worth solving.

Intuition

Given a list of positive values, we are asked to randomly pick up a value based on the weight of each value. To put it simple, the task is to do sampling with weight.

Let us look at a simple example. Given an input list of values [1, 9], when we pick up a number out of it, the chance is that 9 times out of 10 we should pick the number 9 as the answer.

In other words, the probability that a number got picked is proportional to the value of the number, with regards to the total sum of all numbers.

To understand the problem better, let us imagine that there is a line in the space, we then project each number into the line according to its value, i.e. a large number would occupy a broader range on the line compared to a small number. For example, the range for the number 9 should be exactly nine times as the range for the number 1.

![img](https://leetcode.com/problems/random-pick-with-weight/Figures/528/528_throw_ball.png)

Now, let us throw a ball randomly onto the line, then it is safe to say there is a good chance that the ball will fall into the range occupied by the number 9. In fact, if we repeat this experiment for a large number of times, then statistically speaking, 9 out of 10 times the ball will fall into the range for the number 9.

Voila. That is the intuition behind this problem.

Simulation

So to solve the problem, we can simply simulate the aforementioned experiment with a computer program.

First of all, let us construct the line in the experiment by chaining up all values together.

Let us denote a list of numbers as [w_1, w_2, w_3, ..., w_n][w 
1
​
 ,w 
2
​
 ,w 
3
​
 ,...,w 
n
​
 ]. Starting from the beginning of the line, we then can represent the offsets for each range KK as (\sum_{1}^{K}{w_i}, \sum_{1}^{K+1}{w_i})(∑ 
1
K
​
 w 
i
​
 ,∑ 
1
K+1
​
 w 
i
​
 ), as shown in the following graph:
 ![img](https://leetcode.com/problems/random-pick-with-weight/Figures/528/528_prefix_sum_formula.png)

As many of you might recognize now, the offsets of the ranges are actually the prefix sums from a sequence of numbers. For each number in a sequence, its corresponding prefix sum, also known as cumulative sum, is the sum of all previous numbers in the sequence plus the number itself.

As an observation from the definition of prefix sums, one can see that the list of prefix sums would be strictly monotonically increasing, if all numbers are positive.

To throw a ball on the line is to find an offset to place the ball. Let us call this offset target.

Once we randomly generate the target offset, the task is now boiled down to finding the range that this target falls into.

Let us rephrase the problem now, given a list of offsets (i.e. prefix sums) and a target offset, our task is to fit the target offset into the list so that the ascending order is maintained.

**Approach : Prefix Sums with Binary Search**

Intuition

Concerning the above problem, arguably the most intuitive solution would be linear search. Many of you might have already thought one step ahead, by noticing that the input list is sorted, which is a sign to apply a more advanced search algorithm called binary search.

Let us do one thing at one time. In this approach, we will first focus on the linear search algorithm so that we could work out other implementation details. In the next approach, we will then improve upon this approach with a binary search algorithm.

So far, there is one little detail that we haven't discussed, which is how to randomly generate a target offset for the ball. By "randomly", we should ensure that each point on the line has an equal opportunity to be the target offset for the ball.

In most of the programming languages, we have some random() function that generates a random value between 0 and 1. We can scale up this randomly-generated value to the entire range of the line, by multiplying it with the size of the range. At the end, we could use this scaled random value as our target offset.

As an alternative solution, sometimes one might find a randomInteger(range) function that could generate a random integer from a given range. One could then directly use the output of this function as our target offset.

Here, we adopt the random() function, since it could also work for the case where the weights are float values.

Algorithm

We now should have all the elements at hand for the implementation.

First of all, before picking an index, we should first set up the playground, by generating a list of prefix sums from a given list of numbers. The best place to do so would be in the constructor of the class, so that we don't have to generate it again and again at the invocation of pickIndex() function.

In the constructor, we should also keep the total sum of the input numbers, so that later we could use this total sum to scale up the random number.
For the pickIndex() function, here are the steps that we should perform.

Firstly, we generate a random number between 0 and 1. We then scale up this number, which will serve as our target offset.

We then scan through the prefix sums that we generated before by linear search, to find the first prefix sum that is larger than our target offset.

And the index of this prefix sum would be exactly the right place that the target should fall into. We return the index as the result of pickIndex() function.

As a reminder, the condition to apply binary search on a list is that the list should be sorted, either in ascending or descending order. For the list of prefix sums that we search on, this condition is guaranteed.

Algorithm

The only place we need to modify is the pickIndex() function, where we replace the linear search with the binary search.





In [97]:
import random
class Solution:
    def __init__(self, w):
        """
        :type w: List[int]
        """
        self.prefix_sums = []
        prefix_sum = 0
        for weight in w:
            prefix_sum += weight
            self.prefix_sums.append(prefix_sum)
        self.total_sum = prefix_sum

    def pickIndex(self) -> int:
        """
        :rtype: i nt
        """
        target = self.total_sum * random.random()
        # run a binary search to find the target zone
        low, high = 0, len(self.prefix_sums)
        while low < high:
            mid = low + (high - low) // 2
            if target > self.prefix_sums[mid]:
                low = mid + 1
            else:
                high = mid
        return low

# Driver Code
if __name__ == "__main__":
    w = [1,3]
    obj = Solution(w)
    print(obj.pickIndex())
    

1


# 19. Find and Replace in a String

You are given a 0-indexed string s that you must perform k replacement operations on. The replacement operations are given as three 0-indexed parallel arrays, indices, sources, and targets, all of length k.

To complete the ith replacement operation:

Check if the substring sources[i] occurs at index indices[i] in the original string s.
If it does not occur, do nothing.
Otherwise if it does occur, replace that substring with targets[i].
For example, if s = "abcd", indices[i] = 0, sources[i] = "ab", and targets[i] = "eee", then the result of this replacement will be "eeecd".

All replacement operations must occur simultaneously, meaning the replacement operations should not affect the indexing of each other. The testcases will be generated such that the replacements will not overlap.

For example, a testcase with s = "abc", indices = [0, 1], and sources = ["ab","bc"] will not be generated because the "ab" and "bc" replacements overlap.
Return the resulting string after performing all replacement operations on s.

A substring is a contiguous sequence of characters in a string.

 

Example 1:
![img](https://assets.leetcode.com/uploads/2021/06/12/833-ex1.png)
```
Input: s = "abcd", indices = [0, 2], sources = ["a", "cd"], targets = ["eee", "ffff"]
Output: "eeebffff"
Explanation:
"a" occurs at index 0 in s, so we replace it with "eee".
"cd" occurs at index 2 in s, so we replace it with "ffff".
```

**Algorithm**
1. Store a dictionary to store the indexes as key and (source, target) as value.
2. result to store the replaced string.
3. Loop through S, and check if the current index is in the dictionary and the substring at the current index is equal to the source.
    - If yes, add the target to the result.
        - Also, increment the current index by the length of the source.
    - If no, increment the current index by 1, and add the current character to the result.
4. Return the result.

Time Complexity
* Let N = len(S) and M = len(indexes)
* Building lookup ---> O(M)
* Building result ---> O(N)

Space Complexity
* Building lookup ---> O(M)
* Building result ---> O(N)

In [99]:
class Solution:
    def findReplaceString(self, S, indexes, sources, targets):
        # dict to store index and (source, target)
        lookup = {i: (s, t) for i, s, t in zip(indexes, sources, targets)}
        result = ""
        # curr_index_in_S will be scrolled through
        curr_index_in_S = 0
        # go through S
        while curr_index_in_S < len(S):
            # if the char at S[i] is in the lookup and the substring of S at i starts with the source, then replace it with the target
            if curr_index_in_S in lookup and S[curr_index_in_S:].startswith(lookup[curr_index_in_S][0]):
                result += lookup[curr_index_in_S][1]
                # Since we took chunks of S, we can skip moving through this index
                curr_index_in_S += len(lookup[curr_index_in_S][0])
            else:
                result += S[curr_index_in_S]
                curr_index_in_S += 1
        return result

# Driver Code
if __name__ == "__main__":   
    S = "abcd"
    indexes = [0,2]
    sources = ["a","cd"]
    targets = ["eee","ffff"]
    print(Solution().findReplaceString(S, indexes, sources, targets))

eeebffff


# 20. Guess the Word

This is an interactive problem.

You are given an array of unique strings wordlist where wordlist[i] is 6 letters long, and one word in this list is chosen as secret.

You may call Master.guess(word) to guess a word. The guessed word should have type string and must be from the original list with 6 lowercase letters.

This function returns an integer type, representing the number of exact matches (value and position) of your guess to the secret word. Also, if your guess is not in the given wordlist, it will return -1 instead.

For each test case, you have exactly 10 guesses to guess the word. At the end of any number of calls, if you have made 10 or fewer calls to Master.guess and at least one of these guesses was secret, then you pass the test case.

 
```
Example 1:

Input: secret = "acckzz", wordlist = ["acckzz","ccbazz","eiowzz","abcczz"], numguesses = 10
Output: You guessed the secret word correctly.
Explanation:
master.guess("aaaaaa") returns -1, because "aaaaaa" is not in wordlist.
master.guess("acckzz") returns 6, because "acckzz" is secret and has all 6 matches.
master.guess("ccbazz") returns 3, because "ccbazz" has 3 matches.
master.guess("eiowzz") returns 2, because "eiowzz" has 2 matches.
master.guess("abcczz") returns 4, because "abcczz" has 4 matches.
We made 5 calls to master.guess and one of them was the secret, so we pass the test case.
Example 2:

Input: secret = "hamada", wordlist = ["hamada","khaled"], numguesses = 10
Output: You guessed the secret word correctly.
```

1. If word has score x > 0, only those words that share exactly x characters with word can be the solution.

e.g. suppose guess("xyz") = 1.

"abc" cannot be the solution because it shares 0 letters with "xyz". "xyz" has a score of 1 so "abc" must have at least 1 letter wrong, so cannot be the solution.

"zxy" cannot be the solution because it shares 0 letters (in the right place) with "xyz", so it must have at least one letter wrong.

"ayz" cannot be the solution because it shares 2 letters with "xyz". If "ayz" were the solution then "xyz" would have a score of 2.

"abz" can be the solution because it shares 1 letter with "xyz"

We can use this idea to quickly filter the word list:
```python
class Solution:
   def findSecretWord(self, wordlist: List[str], master: "Master") -> None:
       while wordlist:
           word = wordlist.pop()
           matches = master.guess(word)
           # only those words that share exactly x characters with word can be
           # the solution.
           wordlist = [
               other
               for other in wordlist
               if matches == sum(w == o for w, o in zip(word, other))
           ]
```
But this on its own is not sufficient. We need to shrink the list faster for the big test cases. Which requires the second observation.

2. We eliminate more words by choosing words that are more similar to the rest of the wordlist

If guess("xyz") > 0 then we can eliminate words that are dissimilar to "xyz" (see above). But we can elminate words that are similar to "xyz" if guess("xyz") is == 0. For large wordlists, the overwhelming fraction of words in wordlist will have a score of 0. (Fun problem: compute this fraction for a randomly generated wordlist of size N.)

Given guess() returns 0 most of the time for large wordlists, we can, on average eliminate more words per guess by choosing words that are more similar to the rest of
the corpus.

We can do this by sorting the words in order of "similarity to the rest of the corpus". You can do this pretty much any reasonable way and pass the LeetCode testcases. But just as an example, here's a solution in which we assign each letter at each position a "weight" equal to the number of times that letter occurs at that position across entire wordlist. Each word's similarity to the rest of the corpus is then the sum of these weights for its letters.




In [104]:
from collections import Counter
class Solution:
    def findSecretWord(self, wordlist, master: "Master"):
        # e.g. if wordlist = ["xy", "ab", "xz"]
        # then weights = [{x: 2, a: 1}, {b: 1, y:1, z:1}]
        weights = [Counter(word[i] for word in wordlist) for i in range(6)]

        # sort wordlist, least similar to rest of corpus first
        wordlist.sort(key=lambda word: sum(weights[i][c] for i, c in enumerate(word)))
        count = 0
        while wordlist and count < 10:
            # get the word most similar to the rest of the corpus by popping
            # from the *end* of wordlist
            word = wordlist.pop()
            matches = master.guess(word)
            # only those words that share exactly x characters with word can be
            # the solution.
            wordlist = [
                other
                for other in wordlist
                if matches == sum(w == o for w, o in zip(word, other))
            ]
            count += 1
        return word

class Master:
    def __init__(self, secret: str):
        self.secret = secret
    
    def guess(self, word: str) -> int:
        return sum(w == o for w, o in zip(self.secret, word))
    
# Driver Code
if __name__ == "__main__":
    secret = "acckzz"
    wordlist = ["acckzz","ccbazz","eiowzz","abcczz"]
    numguesses = 10
    master = Master(secret)
    print(Solution().findSecretWord(wordlist, master))

acckzz


# 21. Student Attendance Record II

An attendance record for a student can be represented as a string where each character signifies whether the student was absent, late, or present on that day. The record only contains the following three characters:
```
'A': Absent.
'L': Late.
'P': Present.
```
Any student is eligible for an attendance award if they meet both of the following criteria:
```
The student was absent ('A') for strictly fewer than 2 days total.
The student was never late ('L') for 3 or more consecutive days.
```
Given an integer n, return the number of possible attendance records of length n that make a student eligible for an attendance award. The answer may be very large, so return it modulo 109 + 7.

 
```
Example 1:

Input: n = 2
Output: 8
Explanation: There are 8 records with length 2 that are eligible for an award:
"PP", "AP", "PA", "LP", "PL", "AL", "LA", "LL"
Only "AA" is not eligible because there are 2 absences (there need to be fewer than 2).
Example 2:

Input: n = 1
Output: 3
Example 3:

Input: n = 10101
Output: 183236316
```

Approach #1 Brute Force [Time Limit Exceeded]

In the brute force approach, we actually form every possible string comprising of the letters "A", "P", "L" and check if the string is rewardable by checking it against the given criterias. In order to form every possible string, we make use of a recursive gen(string, n) function. At every call of this function, we append the letters "A", "P" and "L" to the input string, reduce the required length by 1 and call the same function again for all the three newly generated strings.

Approach #2 Using Recursive formulae [Time Limit Exceeded]

Algorithm

The given problem can be solved easily if we can develop a recurring relation for it.

Firstly, assume the problem to be considering only the characters LL and PP in the strings. i.e. The strings can contain only LL and PP. The effect of AA will be taken into account later on.

In order to develop the relation, let's assume that f[n]f[n] represents the number of possible rewardable strings(with LL and PP as the only characters) of length nn. Then, we can easily determine the value of f[n]f[n] if we know the values of the counts for smaller values of nn. To see how it works, let's examine the figure below:
![img](https://leetcode.com/problems/student-attendance-record-ii/Figures/552_Student_Attendence_II.PNG)


The above figure depicts the division of the rewardable string of length nn into two strings of length n-1n−1 and ending with LL or PP. The string ending with PP of length nn is always rewardable provided the string of length n-1n−1 is rewardable. Thus, this string accounts for a factor of f[n-1]f[n−1] to f[n]f[n].

For the first string ending with LL, the rewardability is dependent on the further strings of length n-3n−3. Thus, we consider all the rewardable strings of length n-3n−3 now. Out of the four combinations possible at the end, the fourth combination, ending with a LLLL at the end leads to an unawardable string. But, since we've considered only rewardable strings of length n-3n−3, for the last string to be rewardable at length n-3n−3 and unawardable at length n-1n−1, it must be preceded by a PP before the LLLL.

Thus, accounting for the first string again, all the rewardable strings of length n-1n−1, except the strings of length n-4n−4 followed by PLLPLL, can contribute to a rewardable string of length nn. Thus, this string accounts for a factor of f[n-1] - f[n-4]f[n−1]−f[n−4] to f[n]f[n].

Thus, the recurring relation becomes:

f[n] = 2f[n-1] - f[n-4]f[n]=2f[n−1]−f[n−4]

We store all the f[i]f[i] values in an array. In order to compute f[i]f[i], we make use of a recursive function func(n) which makes use of the above recurrence relation.

Now, we need to put the factor of character AA being present in the given string. We know, atmost one AA is allowed to be presnet in a rewardable string. Now, consider the two cases.

No AA is present: In this case, the number of rewardable strings is the same as f[n]f[n].

A single AA is present: Now, the single AA can be present at any of the nn positions. If the AA is present at the i^{th}i 
th
  position in the given string, in the form: "<(i-1) characters>, A, <(n-i) characters>", the total number of rewardable strings is given by: f[i-1] * f[n-i]f[i−1]∗f[n−i]. Thus, the total number of such substrings is given by: \sum_{i=1}^{n} (f[i-1] * f[n-i])∑ 
i=1
n
​
 (f[i−1]∗f[n−i]).


Approach #3 Using Dynamic Programming [Accepted]

Algorithm

In the last approach, we calculated the values of f[i]f[i] everytime using the recursive function, which goes till its root depth everytime. But, we can reduce a large number of redundant calculations, if we use the results obtained for previous f[j]f[j] values directly to obtain f[i]f[i] as f[i] = 2f[i-1] + f[i-4]f[i]=2f[i−1]+f[i−4].


Approach #4 Dynamic Programming with Constant Space [Accepted]

Algorithm

We can observe that the number and position of PP's in the given string is irrelevant. Keeping into account this fact, we can obtain a state diagram that represents the transitions between the possible states as shown in the figure below:
![img](https://leetcode.com/problems/student-attendance-record-ii/Figures/552_State_Diagram.PNG)

This state diagram contains the states based only upon whether an AA is present in the string or not, and on the number of LL's that occur at the trailing edge of the string formed till now. The state transition occurs whenver we try to append a new character to the end of the current string.

Based on the above state diagram, we keep a track of the number of unique transitions from which a rewardable state can be achieved. We start off with a string of length 0 and keep on adding a new character to the end of the string till we achieve a length of nn. At the end, we sum up the number of transitions possible to reach each rewardable state to obtain the required result.

We can use variables corresponding to the states. axlyaxly represents the number of strings of length ii containing xx a'sa 
′
 s and ending with yy l'sl 
′
 s.


In [108]:
class Solution:
    def __init__(self):
        self.M = 10**9 + 7
    def checkRecord(self, n):
        a0l0 = 1
        a0l1 = 0
        a0l2 = 0
        a1l0 = 0
        a1l1 = 0
        a1l2 = 0
        for i in range(n):
            new_a0l0 = (a0l0 + a0l1 + a0l2) % self.M
            new_a0l1 = a0l0 
            new_a0l2 = a0l1
            new_a1l0 = (a0l0 + a0l1 + a0l2 + a1l0 + a1l1 + a1l2) % self.M
            new_a1l1 = a1l0
            new_a1l2 = a1l1
            a0l0 = new_a0l0
            a0l1 = new_a0l1
            a0l2 = new_a0l2
            a1l0 = new_a1l0
            a1l1 = new_a1l1
            a1l2 = new_a1l2
        return (a0l0 + a0l1 + a0l2 + a1l0 + a1l1 + a1l2) % self.M

# Driver Code
if __name__ == "__main__":
    n = 3
    print(Solution().checkRecord(n))

19


# 22. Count Words Obtained After Adding a Letter

You are given two 0-indexed arrays of strings startWords and targetWords. Each string consists of lowercase English letters only.

For each string in targetWords, check if it is possible to choose a string from startWords and perform a conversion operation on it to be equal to that from targetWords.

The conversion operation is described in the following two steps:

Append any lowercase letter that is not present in the string to its end.
For example, if the string is "abc", the letters 'd', 'e', or 'y' can be added to it, but not 'a'. If 'd' is added, the resulting string will be "abcd".
Rearrange the letters of the new string in any arbitrary order.
For example, "abcd" can be rearranged to "acbd", "bacd", "cbda", and so on. Note that it can also be rearranged to "abcd" itself.
Return the number of strings in targetWords that can be obtained by performing the operations on any string of startWords.

Note that you will only be verifying if the string in targetWords can be obtained from a string in startWords by performing the operations. The strings in startWords do not actually change during this process.

 
```
Example 1:

Input: startWords = ["ant","act","tack"], targetWords = ["tack","act","acti"]
Output: 2
Explanation:
- In order to form targetWords[0] = "tack", we use startWords[1] = "act", append 'k' to it, and rearrange "actk" to "tack".
- There is no string in startWords that can be used to obtain targetWords[1] = "act".
  Note that "act" does exist in startWords, but we must append one letter to the string before rearranging it.
- In order to form targetWords[2] = "acti", we use startWords[1] = "act", append 'i' to it, and rearrange "acti" to "acti" itself.
Example 2:

Input: startWords = ["ab","a"], targetWords = ["abc","abcd"]
Output: 1
Explanation:
- In order to form targetWords[0] = "abc", we use startWords[0] = "ab", add 'c' to it, and rearrange it to "abc".
- There is no string in startWords that can be used to obtain targetWords[1] = "abcd".
```

Approach 1: Trie

We sort and store start words in the trie. When matching, we have an option to skip a letter from the target word. This works because there are no repeated characters in target words.

Time:  building the trie: O(s[looping through start words]*mlogm[sorting the word]*m[building a trie]) + O(t[looping in target words]*mlogm[sorting each word]*m[removing a character]*m[searching through trie])

Space: O(sm) for building the trie



In [110]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.end = False

class Solution:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        curr = self.root
        for c in word:
            if c not in curr.children:
                curr.children[c] = TrieNode()
            curr = curr.children[c]
        curr.end = True
    def search(self, word):
        curr = self.root
        for c in word:
            if c not in curr.children:
                return False
            curr = curr.children[c]
        return curr.end
    
    def wordCount(self, startWords, targetWords):
        for word in startWords:
            self.insert(sorted(list(word)))
        
        ans = 0
        for word in targetWords:
            target = sorted(list(word))
            for i in range(len(target)):
                w = target[:i] + target[i+1:]
                if self.search(w):
                    ans += 1
                    break 
        return ans

# Driver Code
if __name__ == "__main__":
    startWords = ["ant","act","tack"]
    targetWords = ["tack","act","acti"]
    print(Solution().wordCount(startWords, targetWords))


2


# 23. Text Justification

Given an array of strings words and a width maxWidth, format the text such that each line has exactly maxWidth characters and is fully (left and right) justified.

You should pack your words in a greedy approach; that is, pack as many words as you can in each line. Pad extra spaces ' ' when necessary so that each line has exactly maxWidth characters.

Extra spaces between words should be distributed as evenly as possible. If the number of spaces on a line does not divide evenly between words, the empty slots on the left will be assigned more spaces than the slots on the right.

For the last line of text, it should be left-justified, and no extra space is inserted between words.

Note:

A word is defined as a character sequence consisting of non-space characters only.
Each word's length is guaranteed to be greater than 0 and not exceed maxWidth.
The input array words contains at least one word.
 
```
Example 1:

Input: words = ["This", "is", "an", "example", "of", "text", "justification."], maxWidth = 16
Output:
[
   "This    is    an",
   "example  of text",
   "justification.  "
]
Example 2:

Input: words = ["What","must","be","acknowledgment","shall","be"], maxWidth = 16
Output:
[
  "What   must   be",
  "acknowledgment  ",
  "shall be        "
]
Explanation: Note that the last line is "shall be    " instead of "shall     be", because the last line must be left-justified instead of fully-justified.
Note that the second line is also left-justified because it contains only one word.
Example 3:

Input: words = ["Science","is","what","we","understand","well","enough","to","explain","to","a","computer.","Art","is","everything","else","we","do"], maxWidth = 20
Output:
[
  "Science  is  what we",
  "understand      well",
  "enough to explain to",
  "a  computer.  Art is",
  "everything  else  we",
  "do                  "
]
```



In [113]:
class Solution:
    def fullJustify(self, words, maxWidth):
            
        result, current_list, num_of_letters = [],[], 0
        # result -> stores final result output
        # current_list -> stores list of words which are traversed but not yet added to result
        # num_of_letters -> stores number of chars corresponding to words in current_list
        
        for word in words:
            
            # total no. of chars in current_list + total no. of chars in current word
            # + total no. of words ~= min. number of spaces between words
            if num_of_letters + len(word) + len(current_list) > maxWidth:
                # size will be used for module "magic" for round robin
                # we use max. 1 because atleast one word would be there and to avoid modulo by 0
                size = max(1, len(current_list)-1)
                
                for i in range(maxWidth-num_of_letters):
                    # add space to each word in round robin fashion
                    index = i%size
                    current_list[index] += ' ' 
                
                # add current line of words to the output
                result.append("".join(current_list))
                current_list, num_of_letters = [], 0
            
            # add current word to the list and add length to char count
            current_list.append(word)
            num_of_letters += len(word)
        
        # form last line by join with space and left justify to maxWidth using ljust (python method)
        # that means pad additional spaces to the right to make string length equal to maxWidth
        result.append(" ".join(current_list).ljust(maxWidth))
        
        return result

# Driver Code
if __name__ == "__main__":
    words = ["This", "is", "an", "example", "of", "text", "justification."]
    maxWidth = 16
    print(Solution().fullJustify(words, maxWidth))

['This    is    an', 'example  of text', 'justification.  ']


# 24. Tiling a Rectangle with the Fewest Squares

Given a rectangle of size n x m, return the minimum number of integer-sided squares that tile the rectangle.
```
Example 1:
Input: n = 2, m = 3
Output: 3
Explanation: 3 squares are necessary to cover the rectangle.
2 (squares of 1x1)
1 (square of 2x2)
```

I've decided to solve this question with backtracking, and here's the basic idea:
1. We'll solve this problem by placing tiles one after another, trying to fill the entire board. We'll start from the top left corner.

2. The process of filling will be pretty simple:

    * Find the next empty cell
    * Find the max tile size we can place at the location
    * Iterate all possible sizes (1-max_size) and try placing them
        * place tile
        * backtrack the updated board
        * remove the previously place tile
3. We'll have a variable holding the min tiles count we found so far. At the beginning of each backtrack func execution we'll check if the board is filled and update the min tile count.

Here's the code for that solution:


In [116]:
class Solution:
    def tilingRectangle(self, n: int, m: int):
        board = [[False for _ in range(m)] for _ in range(n)]
        self.min_n_tiles = float('inf')
        
        def find_next_empty_cell(board):
            # go over the entire board, find the first empty cell and return it
            for i in range(n):
                for j in range(m):
                    if not board[i][j]:
                        return i, j
            
            # if the board is all covered, reutrn -1,-1
            return -1, -1
        
        def get_max_tile_size(board, row, col):
            # we need to measure the min distance from row, col to a tile/wall
            row_dist = col_dist = 1
            
            while col + col_dist < m and not board[row][col + col_dist]: col_dist += 1                       
            while row + row_dist < n and not board[row + row_dist][col]: row_dist += 1                    
            
            return min(row_dist, col_dist)                                 
        
        def cover(board, row, col, tile_size, cover_val):         
            # cover board at specific location with a specific tile, by the cover val argument (True/False)
            for i in range(tile_size):                    
                for j in range(tile_size):
                    board[row + i][col + j] = cover_val                                    
            
        def backtrack(board, n_tiles):          
            # if curr tile count is already the min_tiles we found so far, no need to continue this placing route
            if n_tiles == self.min_n_tiles: 
                return
            
            next_row, next_col = find_next_empty_cell(board)
            
            # if not empty cell -> update min and break
            if next_row == next_col == -1:
                self.min_n_tiles = min(self.min_n_tiles, n_tiles)
                return
            
            # find max tile size to place
            max_tile_size = get_max_tile_size(board, next_row, next_col)
            
            # iterate possible tile size, place and backtrack
            for tile_size in range(max_tile_size, 0, -1):   
                # place tile
                cover(board, next_row, next_col, tile_size, True)
                
                # backtrack
                backtrack(board, n_tiles + 1)
                
                # remove tile
                cover(board, next_row, next_col, tile_size, False)                    


        backtrack(board, 0)
        return self.min_n_tiles

# Driver Code
if __name__ == "__main__":
    n = 4
    m = 5
    print(Solution().tilingRectangle(n, m))

5


```
That's it!
Maybe you've noticed something while going thorugh the code. Using a matrix to represent our board is costly in terms of runtime and space.
Finding the next cell -> O(n*m). Placing and unplacing tile is also a bit messy. Passing the board to each backtrack call is also expensive.

Can we do better? Yes
Instead of matrix, we can maintain an heights array.
heights[i] = respresents the number of cells from the top down, that are covered with tiles

Let's look at an example. n = 2, m = 3.
In the previous solution, we would represent this example with a 2X3 board that would look like that:

[False, False, False]
[False, False, False]
And when we're done covering the board, it would look like that:

[True, True, True]
[True, True, True]
In the new suggested method: heights would look like that: [0, 0, 0]
Let's say we want to place a tile of size 1, we can just update heights to [1, 0, 0]
Or place a tile of size 2 instead -> [2, 2, 0].
Notice that tile of size 2 would update two columns, tile of size 3 would update the next 3 columns and so on.
A board of size n X m would be represent as [0] * m. When it's fully covered, it would look like that: [n] * m because of the m columns contain n covered cells.
```

Here's the code for the new solution:



In [117]:
class Solution:
    
    def tilingRectangle(self, n, m):
        heights = [0] * m
        self.min_tiles = float('inf')
        
        def find_next_location():
            # find the min height, and return the first index of it
            min_height = min(heights)
            return heights.index(min_height)
        
        def find_max_tile_size(col_index):  
            # when we want to check the max tile size we can place, we need to take into account 3 things:
                # 1. we can't go more the m - col_index steps to the right
                # 2. we can't go more than n - heights[col_index] steps down
                # 3. we can't put a tile bigger than the distance to next tile to the right. We can check that by comparing heights of adjacent columns
            curr_height = heights[col_index]
            max_height_dist = n - curr_height
            
            curr_index = col_index + 1
            while curr_index < m and heights[curr_index] == curr_height and curr_index - col_index < max_height_dist:
                curr_index += 1
                        
            return curr_index - col_index
        
        def backtrack(tile_count):
            # no need to go further      
            if tile_count == self.min_tiles: return
            
            # find the next possible location to place a tile
            next_col = find_next_location()
            
            # if the min height is n, the board is fully covered
            if heights[next_col] == n:
                self.min_tiles = min(self.min_tiles, tile_count)
                return
            
            # find the max tile size we can place at the current position
            max_tile_size = find_max_tile_size(next_col) 
                               
            # iterate tile option
            for tile_size in range(max_tile_size, 0, -1):
                
                # place tile
                for col in range(next_col, next_col + tile_size):
                    heights[col] += tile_size
                
                # backtrack
                backtrack(tile_count + 1)
                
                # remove tile
                for col in range(next_col, next_col + tile_size):
                    heights[col] -= tile_size
            
        backtrack(0)
        
        return self.min_tiles

# Driver Code
if __name__ == "__main__":
    n = 4
    m = 5
    print(Solution().tilingRectangle(n, m))
    

5


# 25. Robot Room Cleaner

You are controlling a robot that is located somewhere in a room. The room is modeled as an m x n binary grid where 0 represents a wall and 1 represents an empty slot.

The robot starts at an unknown location in the room that is guaranteed to be empty, and you do not have access to the grid, but you can move the robot using the given API Robot.

You are tasked to use the robot to clean the entire room (i.e., clean every empty cell in the room). The robot with the four given APIs can move forward, turn left, or turn right. Each turn is 90 degrees.

When the robot tries to move into a wall cell, its bumper sensor detects the obstacle, and it stays on the current cell.

Design an algorithm to clean the entire room using the following APIs:
```
interface Robot {
  // returns true if next cell is open and robot moves into the cell.
  // returns false if next cell is obstacle and robot stays on the current cell.
  boolean move();

  // Robot will stay on the same cell after calling turnLeft/turnRight.
  // Each turn will be 90 degrees.
  void turnLeft();
  void turnRight();

  // Clean the current cell.
  void clean();
}
```
Note that the initial direction of the robot will be facing up. You can assume all four edges of the grid are all surrounded by a wall.

 

Custom testing:

The input is only given to initialize the room and the robot's position internally. You must solve this problem "blindfolded". In other words, you must control the robot using only the four mentioned APIs without knowing the room layout and the initial robot's position.

 

Example 1:
![img](https://assets.leetcode.com/uploads/2021/07/17/lc-grid.jpg)
```
Input: room = [[1,1,1,1,1,0,1,1],[1,1,1,1,1,0,1,1],[1,0,1,1,1,1,1,1],[0,0,0,1,0,0,0,0],[1,1,1,1,1,1,1,1]], row = 1, col = 3
Output: Robot cleaned all rooms.
Explanation: All grids in the room are marked by either 0 or 1.
0 means the cell is blocked, while 1 means the cell is accessible.
The robot initially starts at the position of row=1, col=3.
From the top left corner, its position is one row below and three columns right.
Example 2:

Input: room = [[1]], row = 0, col = 0
Output: Robot cleaned all rooms.
 
```

Approach 1: Spiral Backtracking

Concepts to use

Let's use here two programming concepts.

The first one is called constrained programming.

That basically means to put restrictions after each robot move. Robot moves, and the cell is marked as visited. That propagates constraints and helps to reduce the number of combinations to consider.
![img](https://leetcode.com/problems/robot-room-cleaner/Figures/489/489_constraints.png)

The second one called backtracking.

Let's imagine that after several moves the robot is surrounded by the visited cells. But several steps before there was a cell which proposed an alternative path to go. That path wasn't used and hence the room is not yet cleaned up. What to do? To backtrack. That means to come back to that cell, and to explore the alternative path.

![img](https://leetcode.com/problems/robot-room-cleaner/Figures/489/489_backtrack.png)

Intuition

This solution is based on the same idea as maze solving algorithm called right-hand rule. Go forward, cleaning and marking all the cells on the way as visited. At the obstacle turn right, again go forward, etc. Always turn right at the obstacles and then go forward. Consider already visited cells as virtual obstacles.

What to do if after the right turn there is an obstacle just in front ?

Turn right again.

How to explore the alternative paths from the cell ?

Go back to that cell and then turn right from your last explored direction.

When to stop ?

Stop when you explored all possible paths, i.e. all 4 directions (up, right, down, and left) for each visited cell.

Algorithm

Time to write down the algorithm for the backtrack function backtrack(cell = (0, 0), direction = 0).

* Mark the cell as visited and clean it up.

* Explore 4 directions : up, right, down, and left (the order is important since the idea is always to turn right) :

    * Check the next cell in the chosen direction :

        * If it's not visited yet and there is no obtacles :

            * Move forward.

            * Explore next cells backtrack(new_cell, new_direction).

            * Backtrack, i.e. go back to the previous cell.

        * Turn right because now there is an obstacle (or a virtual obstacle) just in front.

![img](https://leetcode.com/problems/robot-room-cleaner/Figures/489/489_implementation.png)


In [119]:

# """
# This is the robot's control interface.
# You should not implement it, or speculate about its implementation
# """
#class Robot:
#    def move(self):
#        """
#        Returns true if the cell in front is open and robot moves into the cell.
#        Returns false if the cell in front is blocked and robot stays in the current cell.
#        :rtype bool
#        """
#
#    def turnLeft(self):
#        """
#        Robot will stay in the same cell after calling turnLeft/turnRight.
#        Each turn will be 90 degrees.
#        :rtype void
#        """
#
#    def turnRight(self):
#        """
#        Robot will stay in the same cell after calling turnLeft/turnRight.
#        Each turn will be 90 degrees.
#        :rtype void
#        """
#
#    def clean(self):
#        """
#        Clean the current cell.
#        :rtype void
#        """

class Solution:       
    def cleanRoom(self, robot):
        """
        :type robot: Robot
        :rtype: None
        """
        def go_back():
            robot.turnRight()
            robot.turnRight()
            robot.move()
            robot.turnRight()
            robot.turnRight()
            
        def backtrack(cell = (0, 0), d = 0):
            visited.add(cell)
            robot.clean()
            # going clockwise : 0: 'up', 1: 'right', 2: 'down', 3: 'left'
            for i in range(4):
                new_d = (d + i) % 4
                new_cell = (cell[0] + directions[new_d][0], \
                            cell[1] + directions[new_d][1])
                
                if not new_cell in visited and robot.move():
                    backtrack(new_cell, new_d)
                    go_back()
                # turn the robot following chosen direction : clockwise
                robot.turnRight()
    
        # going clockwise : 0: 'up', 1: 'right', 2: 'down', 3: 'left'
        directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        visited = set()
        backtrack()


# 26. Swap Adjacent in LR String

In a string composed of 'L', 'R', and 'X' characters, like "RXXLRXRXL", a move consists of either replacing one occurrence of "XL" with "LX", or replacing one occurrence of "RX" with "XR". Given the starting string start and the ending string end, return True if and only if there exists a sequence of moves to transform one string to the other.

 
```
Example 1:

Input: start = "RXXLRXRXL", end = "XRLXXRRLX"
Output: true
Explanation: We can transform start to end following these steps:
RXXLRXRXL ->
XRXLRXRXL ->
XRLXRXRXL ->
XRLXXRRXL ->
XRLXXRRLX
Example 2:

Input: start = "X", end = "L"
Output: false
```

Key observations:
```
There are three kinds of characters, ‘L’, ‘R’, ‘X’.
Replacing XL with LX = move L to the left by one
Replacing RX with XR = move R to the right by one
If we remove all the X in both strings, the resulting strings should be the same.
```
Additional observations:
```
Since a move always involves X, an L or R cannot move through another L or R.
Since an L can only move to the right, for each occurrence of L in the start string, its position should be to the same or to the left of its corresponding L in the end string.
```

![img](https://assets.leetcode.com/users/images/8c1c2572-00a8-4413-8383-ec5d5126c060_1601526675.9118004.png)

And vice versa for the R characters.

Implementation

We first compare two strings with X removed. This checks relative position between Ls and Rs are correct.

Then we find the indices for each occurence of L and check the condition in the above figure. Then we do the same for R.




In [127]:
class Solution:
    def canTransform(self, start, end):
        if len(start) != len(end):
            return False
        if len(start) == 1:
            return start == end
            
        # replace all X with ""
        start_removedX = start.replace("X", "")
        end_removedX = end.replace("X", "")
        if start_removedX != end_removedX:
            return False
        
        # find the indices of L in start and end
        start_L_indices = [i for i, x in enumerate(start) if x == "L"]
        end_L_indices = [i for i, x in enumerate(end) if x == "L"]
        # find the indices of R in start and end
        start_R_indices = [i for i, x in enumerate(start) if x == "R"]
        end_R_indices = [i for i, x in enumerate(end) if x == "R"]

        # return False if index of L in start is less than index of L in end
        for i, j in zip(start_L_indices, end_L_indices):
            if i < j:
                return False
        # return False if index of R in start is greater than index of R in end
        for i, j in zip(start_R_indices, end_R_indices):
            if i > j:
                return False
                
        return True


start = "RXXLRXRXL"
end = "XRLXXRRLX"     
print(Solution().canTransform(start, end))

start ="XXXLXXXXXX"
end ="XXXLXXXXXX"
print(Solution().canTransform(start, end))


True
True


# 27. Range Module

A Range Module is a module that tracks ranges of numbers. Design a data structure to track the ranges represented as half-open intervals and query about them.

A half-open interval [left, right) denotes all the real numbers x where left <= x < right.

Implement the RangeModule class:

RangeModule() Initializes the object of the data structure.
void addRange(int left, int right) Adds the half-open interval [left, right), tracking every real number in that interval. Adding an interval that partially overlaps with currently tracked numbers should add any numbers in the interval [left, right) that are not already tracked.
boolean queryRange(int left, int right) Returns true if every real number in the interval [left, right) is currently being tracked, and false otherwise.
void removeRange(int left, int right) Stops tracking every real number currently being tracked in the half-open interval [left, right).
 
```
Example 1:

Input
["RangeModule", "addRange", "removeRange", "queryRange", "queryRange", "queryRange"]
[[], [10, 20], [14, 16], [10, 14], [13, 15], [16, 17]]
Output
[null, null, null, true, false, true]

Explanation
RangeModule rangeModule = new RangeModule();
rangeModule.addRange(10, 20);
rangeModule.removeRange(14, 16);
rangeModule.queryRange(10, 14); // return True,(Every number in [10, 14) is being tracked)
rangeModule.queryRange(13, 15); // return False,(Numbers like 14, 14.03, 14.17 in [13, 15) are not being tracked)
rangeModule.queryRange(16, 17); // return True, (The number 16 in [16, 17) is still being tracked, despite the remove operation)
```

We make use of the python bisect_left and bisect_right functions. bisect_left returns an insertion index in a sorted array to the left of the search value. bisect_right returns an insertion index in a sorted array to the right of the search value. See the python documentation. To keep track of the start and end values of the ranges being tracked, we use a tracking array of integers. This array consists of a number of sorted pairs of start and end values. So, it always has an even number of elements.

addRange first gets the leftmost insertion index of the left value and the rightmost insertion index of the right value. Then, we check if either of these indexes are even. If the index is even, it means that it is currently outside the range of start and end indexes being tracked. In this case, we include this index to be updated in the tracking array. We then use python array slicing to overwrite the tracking array with the left and right values placed in the correct index. Complexity is O(n).

removeRange first gets the leftmost insertion index of the left value and the rightmost insertion index of the right value. Then, we check if either of these indexes are odd. If the index is odd, it means that it is currently inside the range of start and end indexes being tracked. In this case, we include this index to be updated in the tracking array. We then use python array slicing to overwrite the tracking array with the left and right values placed in the correct index. Complexity is O(n).

queryRange gets the rightmost insertion index of the left value and the leftmost insertion index of the right value. If both these indexes are equal and these indexes are odd, it means the range queried is inside the range of values being tracked. In this case, we return True. Else, we return False. Complexity is O(log n).



In [138]:
import bisect
print(bisect.bisect_left([0, 1], 2))
print(bisect.bisect_right([0, 3], 1))


2
1


In [145]:
class RangeModule:
    
    def __init__(self):
        self.track = []

    def addRange(self, left, right):
        start = bisect.bisect_left(self.track, left)
        end = bisect.bisect_right(self.track, right)
        
        subtrack = []
        if start % 2 == 0:
            subtrack.append(left)
        if end % 2 == 0:
            subtrack.append(right)
        print(f"track before {left}, {right}= {self.track}")
        self.track[start:end] = subtrack
        print(f"track after {left}, {right}= {self.track}")

    def removeRange(self, left, right):
        start = bisect.bisect_left(self.track, left)
        print(f"start = {start}")
        end = bisect.bisect_right(self.track, right)
        print(f"end = {end}")
        
        subtrack = []
        if start % 2 == 1:
            subtrack.append(left)
        if end % 2 == 1:
            subtrack.append(right)
        print(f"track before {left}, {right}= {self.track}")
        self.track[start:end] = subtrack
        print(f"track after {left}, {right}= {self.track}")
		
    def queryRange(self, left, right):
        start = bisect.bisect_right(self.track, left)
        end = bisect.bisect_left(self.track, right)
		
        return start == end and start % 2 == 1

# Driver Code
if __name__ == "__main__":
    r = RangeModule()
    r.addRange(10, 20)
    r.addRange(20, 30)
    r.addRange(30, 40)
    print(r.queryRange(10, 20))
    print(r.queryRange(10, 25))
    print(r.queryRange(25, 35))
    r.removeRange(15, 25)
    print(r.queryRange(10, 20))
    print(r.queryRange(10, 25))
    print(r.queryRange(25, 35))


track before 10, 20= []
track after 10, 20= [10, 20]
track before 20, 30= [10, 20]
track after 20, 30= [10, 30]
track before 30, 40= [10, 30]
track after 30, 40= [10, 40]
True
True
True
start = 1
end = 1
track before 15, 25= [10, 40]
track after 15, 25= [10, 15, 25, 40]
False
False
True


# 28. Longest Increasing Path in a Matrix

Given an m x n integers matrix, return the length of the longest increasing path in matrix.

From each cell, you can either move in four directions: left, right, up, or down. You may not move diagonally or move outside the boundary (i.e., wrap-around is not allowed).

 

Example 1:
![img](https://assets.leetcode.com/uploads/2021/01/05/grid1.jpg)
```
Input: matrix = [[9,9,4],[6,6,8],[2,1,1]]
Output: 4
Explanation: The longest increasing path is [1, 2, 6, 9].
Example 2:
```
![img](https://assets.leetcode.com/uploads/2021/01/27/tmp-grid.jpg)
```
Input: matrix = [[3,4,5],[3,2,6],[2,2,1]]
Output: 4
Explanation: The longest increasing path is [3, 4, 5, 6]. Moving diagonally is not allowed.
Example 3:
```
```
Input: matrix = [[1]]
Output: 1
```

Approach 1 (DFS + Memoization) [Accepted]

Intuition

Cache the results for the recursion so that any subproblem will be calculated only once.

Algorithm

From previous analysis, we know that there are many duplicate calculations in the naive approach.

One optimization is that we can use a set to prevent the repeat visit in one DFS search. This optimization will reduce the time complexity for each DFS to O(mn)O(mn) and the total algorithm to O(m^2n^2)O(m 
2
 n 
2
 ).

Here, we will introduce more powerful optimization, Memoization.

In computing, memoization is an optimization technique used primarily to speed up computer programs by storing the results of expensive function calls and returning the cached result when the same inputs occur again.

In our problem, we recursively call dfs(x, y) for many times. But if we already know all the results for the four adjacent cells, we only need constant time. During our search if the result for a cell is not calculated, we calculate and cache it; otherwise, we get it from the cache directly.

Complexity Analysis:

Time Complexity: O(mn) each cell is visited once.

Space Complexity: O(mn + h) = O(mn).

For each DFS we need O(h) space used by the system stack, where h is the maximum depth of the recursion. In the worst case, O(h) = O(m*n).

Each visited set can have at maximum all cells from the matrix so O(mn)

In [159]:
class Solution:
    def longestIncreasingPath(self, matrix) -> int:
        directions = ((0, 1), (0, -1), (-1, 0), (1, 0))

        self.m = len(matrix)
        self.n = len(matrix[0])

        cache = [[0] * self.n for _ in range(self.m)]
        max_len = 0

        def dfs(i, j):
            if cache[i][j] != 0:
                return cache[i][j]
            res = 1
            # work with neighbors
            for x, y in directions:
                x, y = i + x, j + y
                direction_count = 0
                if 0 <= x < self.m and 0 <= y < self.n:
                    if matrix[x][y] > matrix[i][j]:
                        direction_count = 1 + dfs(x, y)
                res = max(res, direction_count)
            cache[i][j] = res
            return res

        for i in range(self.m):
            for j in range(self.n):
                max_len = max(max_len, dfs(i, j))
        
        
        return max_len
            

# Driver Code
if __name__ == "__main__":
    matrix = [
        [9,9,4],
        [6,6,8],
        [2,1,1]
    ]
    print(Solution().longestIncreasingPath(matrix))

4


# 29. Swim in Rising Water

You are given an n x n integer matrix grid where each value grid[i][j] represents the elevation at that point (i, j).

The rain starts to fall. At time t, the depth of the water everywhere is t. You can swim from a square to another 4-directionally adjacent square if and only if the elevation of both squares individually are at most t. You can swim infinite distances in zero time. Of course, you must stay within the boundaries of the grid during your swim.

Return the least time until you can reach the bottom right square (n - 1, n - 1) if you start at the top left square (0, 0).

 

Example 1:

![img](https://assets.leetcode.com/uploads/2021/06/29/swim1-grid.jpg)
```
Input: grid = [[0,2],[1,3]]
Output: 3
Explanation:
At time 0, you are in grid location (0, 0).
You cannot go anywhere else because 4-directionally adjacent neighbors have a higher elevation than t = 0.
You cannot reach point (1, 1) until time 3.
When the depth of water is 3, we can swim anywhere inside the grid.
```

Approach #1: Heap [Accepted]

Intuition and Algorithm

Let's keep a priority queue of which square we can walk in next. We always walk in the smallest one that is 4-directionally adjacent to ones we've visited.

When we reach the target, the largest number we've visited so far is the answer.

Complexity Analysis

Time Complexity: O(N^2 \log N)O(N 
2
 logN). We may expand O(N^2)O(N 
2
 ) nodes, and each one requires O(\log N)O(logN) time to perform the heap operations.

Space Complexity: O(N^2)O(N 
2
 ), the maximum size of the heap.




In [160]:
class Solution(object):
    def swimInWater(self, grid):
        N = len(grid)

        seen = {(0, 0)}
        pq = [(grid[0][0], 0, 0)]
        ans = 0
        while pq:
            d, r, c = heapq.heappop(pq)
            ans = max(ans, d)
            if r == c == N-1: return ans
            for cr, cc in ((r-1, c), (r+1, c), (r, c-1), (r, c+1)):
                if 0 <= cr < N and 0 <= cc < N and (cr, cc) not in seen:
                    heapq.heappush(pq, (grid[cr][cc], cr, cc))
                    seen.add((cr, cc))

# Driver Code
if __name__ == "__main__":
    grid = [[0,2],[1,3]]
    print(Solution().swimInWater(grid))
    

3


# 30. Minimum Area Rectangle

You are given an array of points in the X-Y plane points where points[i] = [xi, yi].

Return the minimum area of a rectangle formed from these points, with sides parallel to the X and Y axes. If there is not any such rectangle, return 0.

Example 1:
![img](https://assets.leetcode.com/uploads/2021/08/03/rec1.JPG)
```
Input: points = [[1,1],[1,3],[3,1],[3,3],[2,2]]
Output: 4
```

Optimal Approach (Using HashMap) :

We can store values in a structured manner to reduce the number of iterations.
One interesting obsevation about the rectangle having sides parallel to x & y axis is :
Given two points of a diagonal, we can calculate the remaining two points of the second diagonal.

![img](https://assets.leetcode.com/users/images/ae788e03-3252-4f57-b2d2-9567b9cc6a49_1632377423.275515.png)

Suppose we fix two points A & B out of N points in a plane, its possible to have AB as a diagonal of a rectangle only if
the x & y coordinates of points A & B are different ie. x1 is not equal to x2 AND y1 is not equal to y2.

Having A & B fixed, we can calculate coordinates of points C & D which can possibly form a rectangle.
x coordinate of C = x coordinate of B = x2
y coordinate of C = y coordinate of A = y1
x coordinate of D = x coordinate of A = x1
y coordinate of D = y coordinate of B = y2
The above coordinates are the possible coordinates of points C & D.
Now we have to search for the points (out of all the points) having same coordinates as that of the calculated possible coordinates.

So far, our approach is to iterate through different (A,B) pairs and for each pair, we have to search for points C & D.
Speaking of Time Complexity :
O(N^2) [iterate through different (A,B) pairs] x O(N^2) [iterate through all possible (C,D) points].
Which still makes the Time Complexity as O(N^4).
Basically for figuring out points C & D out of the remaining points, we are iterating through all the remaining points which is same as the brute force approach.
This gives us an intuition that we need to somehow eliminate few set of points without iterating through each of them.
![img](https://assets.leetcode.com/users/images/733c7645-0baa-4564-a3e1-6ec91c0cc9e0_1632377473.5340226.png)

In the above diagram, P1, P2, P3 & P4 all have x coordinate same as of point A. Altough each of them have different y coordinates.
But only P1 is resposible for the formation of rectangle because the y coordinate of P1 is same as that of B.

Hence we quickly found out a point (out of 4 points) that was having x coordinate of A & y coordinate of B.
This idea can be implemented using a HashMap where the lookup time will be of O(1) on average.
So if we group the y coordinates on the basis of same x coordinate, using a HashSet we can search for corresponding y coordinate in O(1) time.

So lets create a HashMap having

Key => Different x coordinates

Value => HashSet of y coordinates of points having same x coordinate

So while iterating through different (A,B) pairs, we can check whether there exists value cooresponding to C & D points in the hashmap.
Lets say we check whether there exists a point D in the hashmap,
Step 1 : Key => x coordinate of point A (x1)
Step 2 : Value => Corresponding to Key(x1), search for y coordinate of point B (y2) in HashSet
If coordinates of D exists in the HashMap, then a rectangle is possible.
If coordinates of D doesn't exists in the HashMap, then a rectangle is NOT possible, and we iterate to a different (A,B) pair.

In [163]:
class Solution(object):
    def minAreaRect(self, points):
        """
        :type points: List[List[int]]
        :rtype: int
        """
        table={}
        minimum=float('inf')
        
        # adding all the y co-ordinates with the x co-o as key
        for i in range(len(points)):
            if points[i][0] not in table:
                table[points[i][0]]=[points[i][1]]
            else:
                table[points[i][0]].append(points[i][1])

        # checking for all combinations of points A=(x1,y1), B=(x2,y2)
        for i in range(len(points)):
            x1=points[i][0]
            y1=points[i][1]
            for j in range(len(points)):
                x2=points[j][0]
                y2=points[j][1]
                
                # A and B are in diagonal position, not in the same line
                if x1>x2 and y1>y2:
                    # there exists C=(x1,y2) and D=(x2,y1)
                    if y2 in table[x1] and y1 in table[x2]:
                        minimum=min(minimum, abs(x1-x2) * abs(y1-y2))
        
        # if no such rectangle exists return 0
        if minimum==float('inf'):
            return 0
        return minimum

# Driver Code
if __name__ == "__main__":
    points = [[1,1],[1,3],[3,1],[3,3],[2,2]]
    print(Solution().minAreaRect(points))

4


In [164]:
import heapq
import math

class Solution:
    def minAreaRect(self, points) -> int:
        min_area = math.inf
        q = [x for x in points]
        heapq.heapify(q)
        
        max_y_line = {}
        
        while len(q) > 0:
            top = q[0][0]
            ps = []
            
            # get all the (x,y) pairs for this x
            while len(q) > 0 and q[0][0] == top:
                ps.append(heapq.heappop(q))
            
            lines = []
            
            if len(ps) > 1:
                x = ps[0][0]
                
                # generate all the combinations of left side lines
                for i in range(len(ps)-1):
                    for j in range(i+1, len(ps)):
                        lines.append((ps[i][1], ps[j][1]))
                
                # for all the combos, if a line has occurred before calculate the area
                for line in lines:
                    if line in max_y_line:
                        xmin = max_y_line[line]
                        min_area = min(min_area, self.area(xmin, x, line[0], line[1]))
                    max_y_line[line] = x
                    
        return min_area if min_area != math.inf else 0
    
    def area(self, x1, x2, y1, y2):
        return abs(x2-x1)*abs(y2-y1)

# Driver Code
if __name__ == "__main__":
    points = [[1,1],[1,3],[3,1],[3,3],[2,2]]
    print(Solution().minAreaRect(points))

4


# 31. The number of weak characters in the Game

You are playing a game that contains multiple characters, and each of the characters has two main properties: attack and defense. You are given a 2D integer array properties where properties[i] = [attacki, defensei] represents the properties of the ith character in the game.

A character is said to be weak if any other character has both attack and defense levels strictly greater than this character's attack and defense levels. More formally, a character i is said to be weak if there exists another character j where attackj > attacki and defensej > defensei.

Return the number of weak characters.

 
```
Example 1:

Input: properties = [[5,5],[6,3],[3,6]]
Output: 0
Explanation: No character has strictly greater attack and defense than the other.
Example 2:

Input: properties = [[2,2],[3,3]]
Output: 1
Explanation: The first character is weak because the second character has a strictly greater attack and defense.
Example 3:

Input: properties = [[1,5],[10,4],[4,3]]
Output: 1
Explanation: The third character is weak because the second character has a strictly greater attack and defense.
```




In [165]:
class Solution:
    def numberOfWeakCharacters(self, properties):
        
        properties.sort(key=lambda x: (-x[0],x[1]))
        
        ans = 0
        curr_max = 0
        
        for _, d in properties:
            if d < curr_max:
                ans += 1
            else:
                curr_max = d
        return ans

# Driver Code
if __name__ == "__main__":
    properties = [[1,1],[2,2],[3,3]]
    print(Solution().numberOfWeakCharacters(properties))



2


# 32. Minimum number of flips to convert a binart matrix to a zero matrix

Given a m x n binary matrix mat. In one step, you can choose one cell and flip it and all the four neighbors of it if they exist (Flip is changing 1 to 0 and 0 to 1). A pair of cells are called neighbors if they share one edge.

Return the minimum number of steps required to convert mat to a zero matrix or -1 if you cannot.

A binary matrix is a matrix with all cells equal to 0 or 1 only.

A zero matrix is a matrix with all cells equal to 0.

 

Example 1:
![img](https://assets.leetcode.com/uploads/2019/11/28/matrix.png)
```
Input: mat = [[0,0],[0,1]]
Output: 3
Explanation: One possible solution is to flip (1, 0) then (0, 1) and finally (1, 1) as shown.
Example 2:

Input: mat = [[0]]
Output: 0
Explanation: Given matrix is a zero matrix. We do not need to change it.
Example 3:

Input: mat = [[1,0,0],[1,0,0]]
Output: -1
Explanation: Given matrix cannot be a zero matrix.
```


In [168]:
class Solution:
    def minFlips(self, mat):
        '''
        Flatten the matrix into a string, so we can use it as a state, then
        use BFS to find the target which would be '0000..000' depending on the size of the matrix.
        
        Nothing special. This problem is similar to 773. Sliding Puzzle.
        '''
        rows, cols = len(mat), len(mat[0])
        initial = ''.join(str(cell) for row in mat for cell in row)
        print(f"initial: {initial}")
        target = '0' * (rows * cols)
        '''bfs'''
        flips = { '1': '0', '0': '1' }
        def flip(node, pos):
            node[pos] = flips[node[pos]]
            if pos % cols != 0:
                left = pos - 1
                node[left] = flips[node[left]]
            if pos % cols < cols - 1:
                right = pos + 1
                node[right] = flips[node[right]]
            if pos >= cols:
                top = pos - cols
                node[top] = flips[node[top]]
            if pos < (rows - 1) * cols:
                bottom = pos + cols
                node[bottom] = flips[node[bottom]]
        
        q = collections.deque([initial])
        steps = 0
        visited = set()
        while q:
            for _ in range(len(q)):
                node = q.popleft()
                if node == target:
                    return steps
                if node in visited:
                    continue
                visited.add(node)
                for i in range(len(node)):
                    nextNode = list(node)
                    flip(nextNode, i)
                    q.append(''.join(nextNode))
            steps += 1

        return -1

# Driver Code
if __name__ == "__main__":
    mat = [[0,0],[0,1]]
    print(Solution().minFlips(mat))


initial: 0001
3


# 33. Rank Transform of a Matrix

Given an m x n matrix, return a new matrix answer where answer[row][col] is the rank of matrix[row][col].

The rank is an integer that represents how large an element is compared to other elements. It is calculated using the following rules:

The rank is an integer starting from 1.
If two elements p and q are in the same row or column, then:
If p < q then rank(p) < rank(q)
If p == q then rank(p) == rank(q)
If p > q then rank(p) > rank(q)
The rank should be as small as possible.
The test cases are generated so that answer is unique under the given rules.

 

Example 1:
```
Input: matrix = [[1,2],[3,4]]
Output: [[1,2],[2,3]]
Explanation:
The rank of matrix[0][0] is 1 because it is the smallest integer in its row and column.
The rank of matrix[0][1] is 2 because matrix[0][1] > matrix[0][0] and matrix[0][0] is rank 1.
The rank of matrix[1][0] is 2 because matrix[1][0] > matrix[0][0] and matrix[0][0] is rank 1.
The rank of matrix[1][1] is 3 because matrix[1][1] > matrix[0][1], matrix[1][1] > matrix[1][0], and both matrix[0][1] and matrix[1][0] are rank 2.

```

Example 3:
![img](https://assets.leetcode.com/uploads/2020/10/18/rank3.jpg)
```
Input: matrix = [[20,-21,14],[-19,4,19],[22,-47,24],[-19,4,19]]
Output: [[4,2,3],[1,3,4],[5,1,6],[1,3,4]]
```

Overview

This problem is an extension of the original problem, Rank Transform of an Array. However, the original method in the original problem does not work. To tackle this, we need to add some similar methods used in Most Stones Removed with Same Row or Column. Moreover, to avoid Time Limit Exceeded, some optimization tricks should be applied.

It's indeed a hard problem, so please don't be frustrated if you can not solve it.

Below, we will discuss three approaches: Sorting + BFS, Sorting + DFS, and Sorting + Union-Find.

It's recommended to start reading from approach 1. Also, it's a long article, so take your time to read it.

## Approach 1: Sorting + BFS


Intuition

Let's recall the method used in the original Rank Transform of an Array. The idea is simple: sort the values in the array, and arrange the ranks from the lowest value to the highest value.

It's natural to consider applying the same thing to our matrix: sort the values in the matrix, and arrange the ranks from the lowest one to the highest one.

However, this method does not work. In this problem, we are only required to rank values according to row and column, and not to the whole matrix. The condition is looser.

If we arrange the ranks according to the whole matrix, the resulting rank will be huger than what we want.

For example, consider this case:
![ig](https://leetcode.com/problems/rank-transform-of-a-matrix/Figures/5156/5156_1.png)

In this case, if we just rank [2, 3, 4, 5] by their values as [1, 2, 3, 4], we will get a larger rank for some elements.

We need to make some modifications to get our solution to work.

The idea of sorting and ranking from small value to large value is good. The only problem is that the rank is larger than required. We want to reduce the rank to as small as possible.

When arranging ranks, we can check existing ranks in the same row and the same column, and let the rank be the largest rank checked plus one.

For example, in the above matrix, when we fill in the rank of value 4 (corresponding order is 3):
![img](https://leetcode.com/problems/rank-transform-of-a-matrix/Figures/5156/5156_2.png)

The pseudo-code is as below. Let the required rank matrix be answer.
```
initial answer to all zero
for (i, j) in sorted_order:
    rank = -1
    for row in 0...m-1:
        rank = max(rank, answer[row][j] + 1)
    for col in 0...n-1:
        rank = max(rank, answer[i][col] + 1)
    answer[i][j] = rank
```
However, this approach still can not achieve the target rank matrix. There are two problems, and we will discuss that later.

Now, let's analyze the complexity first.
```
It's recommended to find out an entire working approach and then optimize it. However, for the convenience of writing, we do the optimization here.
```
Let MM be the number of rows and NN be the number of columns.

Since there is \mathcal{O}(NM)O(NM) points in the matrix, and for each point, we are required to search the row and column to determine its rank, the overall time complexity is \mathcal{O}(NM\cdot(N+M))O(NM⋅(N+M)).

In the worst cases, where M=500M=500 and N=500N=500, NM\cdot(N+M) = 500 \cdot 500 \cdot (500 + 500) = 2.5 * 10^8NM⋅(N+M)=500⋅500⋅(500+500)=2.5∗10 
8
 .

Generally, to avoid Time Limit Exceeded, a complexity less than 10^910 
9
  is needed. 10^810 
8
  is dangerous. Can we simplify it?

Notice that we calculated the max of each row and each column many times. We can pre-calculate the maximum before iteration and update that max during the iteration.

We can use two arrays, rowMax and colMax, to record the maximum rank of each row and each column, respectively.
```
rowMax[i] means the max rank in i row, and colMax[j] means the max rank in j column.
```
Take the above example again. If we use these two arrays, we calculate ranks in this way:
![img](https://leetcode.com/problems/rank-transform-of-a-matrix/Figures/5156/5156_3.png)

The pseudo-code is as below.
```
initial answer to all zero
initial rowMax and colMax to all zero
for (i, j) in sorted_order:
    rank = max(rowMax[i], colMax[i]) + 1
    answer[i][j] = rank
    update rowMax and colMax
```
Good. Now we only need \mathcal{O}(1)O(1) for each point. The overall complexity is \mathcal{O}(NM)O(NM) for this part. Notice that sorting requires \mathcal{O}(NM\log(NM))O(NMlog(NM)), so the complexity so far is \mathcal{O}(NM\log(NM))O(NMlog(NM)).

Go back to our approach. In the above, we mention that there are two problems in the code.

The first one is that the minimal rank is not always the maximum of other ranks in the same row and columns plus one. It might be the same as the maximum.

For instance, consider this case:
![img](https://leetcode.com/problems/rank-transform-of-a-matrix/Figures/5156/5156_4.png)

We can see that, if the value is the same, we may not need to add one to the maximum rank.

Well... we can use some if-conditions to solve this problem.

The second problem is even worse: we may need to adjust the previous rank we set before.

Take the below case as an example. In this case, we have filled all the rank matrix except the right-down corner.
![img](https://leetcode.com/problems/rank-transform-of-a-matrix/Figures/5156/5156_5.png)

Since points with the same value in the same row or same column should share the same rank, we need to adjust the other 33.

So, how to solve this problem?

Let's dig into what parts should be adjusted.

Consider this case:
![img](https://leetcode.com/problems/rank-transform-of-a-matrix/Figures/5156/5156_6.png)

Note that the connected points should share the same rank, since they are connected by some "same row or same column" connections.

Also, there is one single 11 and one single 33 that do not connect to any other points, since they do not have such connection.

In conclusion, the points with the same value connected by the "same row or same column" connection should share the same rank. Let's call those points "connected part".

Connected part means a group of points with the same value, where any two points can be linked by a path consisting of horizontal lines ("the same row" connection) and vertical lines ("the same column" connection).

To avoid adjusting, we can find out the whole connected part's maximum rank first and then update that rank to each point in the part.

In this way, we can also avoid the first problem because the ranks in different connected parts are always different.

Now the question remaining is how to find the "connected parts"?

This question is similar to Most Stones Removed with Same Row or Column, where we need to find the numbers of connected parts.

In Most Stones Removed with Same Row or Column, there are three methods to locate the connected parts: BFS, DFS, and Union-Find. Here, we discuss BFS first.

In approach 1, we discuss BFS first and discuss the remaining two in approach 2 and approach 3, respectively.

The idea of BFS is simple: from a starting point, add all the directly connected points (i.e., with the same row or same column) into a waiting queue, pop points from the waiting queue, add new directly connected points into the queue, and repeat until the queue is empty.
```
add starting points to Queue q
while q is not empty:
    point p = q.pop()
    add p's adjacent points to q, skip visited points
```
This search costs \mathcal{O}(V + E)O(V+E), where VV is the number of points and EE is the number of edges in the graph, since we visit each point and each edge constant times.

We have \mathcal{O}(NM)O(NM) points, and if every two points are connected, the number of edges are \mathcal{O}((NM)^2)O((NM) 
2
 ). Therefore, in worst case, \mathcal{O}(V + E) = \mathcal{O}((NM)^2) = 500^4 = 6.25 * 10^{10}O(V+E)=O((NM) 
2
 )=500 
4
 =6.25∗10 
10
 . This will absolutely cause Time Limit Exceeded.

Can we simplify it?

Instead of connecting point to point, we can connect row to column, and column to row.

Consider viewing a point (i, j) as an edge linking i-th row and j-th column.

For example, see this case:

![img](https://leetcode.com/problems/rank-transform-of-a-matrix/Figures/5156/5156_7.png)

For a point (i, j), we connect i-th row and j-th column together. With this graph, we can easily find the connected parts.
```
For example, in the graph above, starting from (0, 0), searching the neighbors of Row 0, we can find Col 0, and Col 2. Therefore, (0, 0) and (0, 2) are connected.

Continue searching the neighbors of Col2, we can find Row 0 and Row 2. We have visited Row 0, but Row 2 is new. Hence, (0, 0), (0, 2), and (2, 2) are connected. Search the neighbors of Row 2, we find nothing new. For (0, 0), we can also search the neighbors of Col 0.

After this search, we get (0, 0)'s connected parts: (0, 0), (0, 2), and (2, 2).
```
Now we need to store the graph. We can have a map graphRow, where graphRow[i] represent the columns linked to i-th row, and a map graphCol, where graphCol[j] represent the columns linked to j-th col.

Wait a minute. Can we combine those two maps into a single map?

Note that the indexes of row start from 0 and occupy positive numbers. There are negative numbers that remain unused. We can store indexes of columns in negative numbers.

A natural idea is to use the negative of column index -\text{col}−col. But both row and column indexes use zero, resulting in duplication of number zero. We need to shift one unit to avoid repetition: using -\text{col} - 1−col−1.

Luckily, we happen to have an operator called "complement" (\sim∼), where \sim\text{col} = -\text{col} - 1∼col=−col−1. What's more, simple math shows \sim(\sim\text{col}) = \text{col}∼(∼col)=col.

Therefore, we can use a single graph to store the connections between row and column: if i >= 0, graph[i] represents i-th row's neighbors (the complement of indexes of linked columns), and if i < 0, graph[~i] gives ~i-th column's adjacent points (the indexes of linked rows).
```
It's also OK to use two single maps to represent the connection relationship. People just use this trick to make implementation a bit simpler.
```
Now, only MM points (represent rows) and NN points (represent columns) are in the graph. Since we can not have edges between rows or between columns, the largest number of edges are \mathcal{O}(NM) = 2.5 * 10^5O(NM)=2.5∗10 
5
 . The number is small enough to pass the test.

So, we successfully achieved finding connected parts by BFS. The remaining part is to fill our rank matrix answer by connected parts, in the sorted value order.
```
initial answer to all zero
initial rowMax and colMax to all zero
for connected_part in sorted_connected_parts:
    rank = -1
    for point (i, j) in connected_part:
        rank = (rank, max(rowMax[i], colMax[i]) + 1)
    for point (i, j) in connected_part:
        answer[i][j] = rank
        update rowMax and colMax
```

By far, we solve every problem we encounter and cleverly avoid Time Limit Exceeded by some optimization. The essence of this algorithm is to separate points into different connected parts, sort them by values, and finally fill in the rank matrix from the lowest value to the highest value.

For the detail of the algorithm, check the "Algorithm" part.

Algorithm

For convenience,
```
We refer "points" to indexes in the matrix, in the form (row_number, column_number).
We refer "value" to the values in the matrix. In other words, the value of point (i, j) is matrix[i][j].
We say two points are "connected" if and only if they have the same values and are in the same row or column, or they are all connected to the same point.
A "connected part" represents a group of connected points.
```

Step 1: Initialize graphs for different values. Iterate matrix and link the rows and columns in the corresponding graph.

Step 2: Initialize a value2index map to store connected parts.
```
This map will contain the value - index mapping. In the index part, separate points to put the connected points in the same array, and to put non-connected points in different arrays. (one array represents a connected part.)
Therefore, value2index should be in this form: {v1: [[point1, point2, ...], [point11, point12, ...], ...], v2: ...}, where point1, point2, ... are connected, and point11, point21, ... are also connected, but none of points from different array are connected.
```
Step 3: Fill in value2index map by iterating over matrix again.
```
For each point, use BFS to find out all the other connected points. Put all of them into value2index as an array.
Remember to mark those points visited to avoid duplicate additions.
```
Step 4: Sort the keys in value2index (i.e., all values in matrix).

Step 5: Initialize our answer matrix. Iterate value2index in the order of sorted keys to fill in answer.
```
For a given key (i.e., a value in matrix), we fill in answer by connected parts (i.e., one array).
Note that for points in the same connected part, they share the same rank.
For a connected part, Find out the minimum possible rank and update that rank.
To reduce the time for searching the minimum possible rank, we need two arrays to record the maximum rank of each row and each column, respectively.
```
Step 6: Return answer.


Complexity Analysis
```
Let MM be the number of rows in matrix and NN be the number of columns in matrix.

Time Complexity: \mathcal{O}(NM\log(NM))O(NMlog(NM)).

We need \mathcal{O}(NM)O(NM) to iterate matrix to build graphs.
We need \mathcal{O}(NM)O(NM) to iterate matrix to build value2index. We only visit points at most twice, since we skip points visited in BFS.
We need \mathcal{O}(NM\log(NM))O(NMlog(NM)) to sort the keys in value2index, since there are at most \mathcal{O}(NM)O(NM) different keys.
We need \mathcal{O}(NM)O(NM) to iterate value2index to build answer.
Adding together, the time we needed is \mathcal{O}(NM\log(NM))O(NMlog(NM)).
Space Complexity: \mathcal{O}(NM)O(NM).

For graphs, we store \mathcal{O}(NM)O(NM) edges (viewing each point as an edge).
For value2index, we store \mathcal{O}(NM)O(NM) points.
For rowMax and columnMax, they have size of \mathcal{O}(M)O(M) and \mathcal{O}(N)O(N), respectively.
In total, the size we needed is \mathcal{O}(NM)O(NM).
```

In [171]:
class Solution:
    def matrixRankTransform(self, matrix):
        m = len(matrix)
        n = len(matrix[0])

        # link row to col, and link col to row
        graphs = {}  # graphs[v]: the connection graph of value v
        for i in range(m):
            for j in range(n):
                v = matrix[i][j]
                # if not initialized, initial it
                if v not in graphs:
                    graphs[v] = {}
                if i not in graphs[v]:
                    graphs[v][i] = []
                if ~j not in graphs[v]:
                    graphs[v][~j] = []
                # link i to j, and link j to i
                graphs[v][i].append(~j)
                graphs[v][~j].append(i)

        # put points into `value2index` dict, grouped by connection
        value2index = {}  # {v: [[points1], [points2], ...], ...}
        seen = set()  # mark whether put into `value2index` or not
        for i in range(m):
            for j in range(n):
                if (i, j) in seen:
                    continue
                seen.add((i, j))
                v = matrix[i][j]
                graph = graphs[v]
                # start bfs
                q = [i, ~j]
                rowcols = {i, ~j}  # store visited row and col
                while q:
                    node = q.pop(0)
                    for rowcol in graph[node]:
                        if rowcol not in rowcols:
                            q.append(rowcol)
                            rowcols.add(rowcol)
                # transform rowcols into points
                points = set()
                for rowcol in rowcols:
                    for k in graph[rowcol]:
                        if k >= 0:
                            points.add((k, ~rowcol))
                            seen.add((k, ~rowcol))
                        else:
                            points.add((rowcol, ~k))
                            seen.add((rowcol, ~k))
                if v not in value2index:
                    value2index[v] = []
                value2index[v].append(points)

        answer = [[0]*n for _ in range(m)]  # the required rank matrix
        rowmax = [0] * m  # rowmax[i]: the max rank in i row
        colmax = [0] * n  # colmax[j]: the max rank in j col
        for v in sorted(value2index.keys()):
            # update by connected points with same value
            for points in value2index[v]:
                rank = 1
                for i, j in points:
                    rank = max(rank, max(rowmax[i], colmax[j]) + 1)
                for i, j in points:
                    answer[i][j] = rank
                    # update rowmax and colmax
                    rowmax[i] = max(rowmax[i], rank)
                    colmax[j] = max(colmax[j], rank)

        return answer

# Driver Code
if __name__ == "__main__":
    matrix = [[1,2],[3,4]]
    print(Solution().matrixRankTransform(matrix))

[[1, 2], [2, 3]]


## Approach 2: Sorting + DFS
```
Intuition

DFS is similar to BFS but differs in the order of searching. In most cases, when the search space is not huge, you can replace BFS with DFS.

In approach 1, we used BFS to find out the connected parts of each point. Now, we use DFS instead.

Algorithm

Step 1: Initialize graphs for different values. Iterate matrix and link the rows and columns in the corresponding graph.

Step 2: Initialize a value2index map to store connected parts.

This map will contain the value - index mapping. In the index part, separate points to put the connected points in the same array, and to put non-connected points in different arrays. (one array represents a connected part.)
Therefore, value2index should be in this form: {v1: [[point1, point2, ...], [point11, point12, ...], ...], v2: ...}, where point1, point2, ... are connected, and point11, point21, ... are also connected, but none of the points from different array are connected.
Step 3: Fill in value2index map by iterating over the matrix again.

For each point, use DFS to find out all the other connected points. Put all of them into value2index as an array.
Remember to mark those points visited to avoid duplicate additions.
Step 4: Sort the keys in value2index (i.e., all values in matrix).

Step 5: Initialize our answer matrix. Iterate value2index in the order of sorted keys to fill in answer.

For a given key (i.e., a value in matrix), we fill in answer by connected parts (i.e., one array).
Note that for points in the same connected part, they share the same rank.
For a connected part, Find out the minimum possible rank and update that rank.
To reduce the time for searching the minimum possible rank, we need two arrays to record the maximum rank of each row and each column, respectively.
Step 6: Return answer.
```


In [172]:
class Solution:
    def matrixRankTransform(self, matrix):
        m = len(matrix)
        n = len(matrix[0])

        # link row to col, and link col to row
        graphs = {}  # graphs[v]: the connection graph of value v
        for i in range(m):
            for j in range(n):
                v = matrix[i][j]
                # if not initialized, initial it
                if v not in graphs:
                    graphs[v] = {}
                if i not in graphs[v]:
                    graphs[v][i] = []
                if ~j not in graphs[v]:
                    graphs[v][~j] = []
                # link i to j, and link j to i
                graphs[v][i].append(~j)
                graphs[v][~j].append(i)

        # put points into `value2index` dict, grouped by connection
        value2index = {}  # {v: [[points1], [points2], ...], ...}
        seen = set()  # mark whether put into `value2index` or not

        def dfs(node, graph, rowcols):
            rowcols.add(node)
            for rowcol in graph[node]:
                if rowcol not in rowcols:
                    dfs(rowcol, graph, rowcols)

        for i in range(m):
            for j in range(n):
                if (i, j) in seen:
                    continue
                seen.add((i, j))
                v = matrix[i][j]
                graph = graphs[v]
                # use dfs to find the connected parts
                rowcols = set()   # store visited row and col
                dfs(i, graph, rowcols)
                dfs(~j, graph, rowcols)
                # transform rowcols into points
                points = set()
                for rowcol in rowcols:
                    for k in graph[rowcol]:
                        if k >= 0:
                            points.add((k, ~rowcol))
                            seen.add((k, ~rowcol))
                        else:
                            points.add((rowcol, ~k))
                            seen.add((rowcol, ~k))
                if v not in value2index:
                    value2index[v] = []
                value2index[v].append(points)

        answer = [[0]*n for _ in range(m)]  # the required rank matrix
        rowmax = [0] * m  # rowmax[i]: the max rank in i row
        colmax = [0] * n  # colmax[j]: the max rank in j col
        for v in sorted(value2index.keys()):
            # update by connected points with same value
            for points in value2index[v]:
                rank = 1
                for i, j in points:
                    rank = max(rank, max(rowmax[i], colmax[j]) + 1)
                for i, j in points:
                    answer[i][j] = rank
                    # update rowmax and colmax
                    rowmax[i] = max(rowmax[i], rank)
                    colmax[j] = max(colmax[j], rank)

        return answer

# Driver Code
if __name__ == "__main__":
    matrix = [[1,2],[3,4]]
    print(Solution().matrixRankTransform(matrix))


[[1, 2], [2, 3]]


# 34. Minimum Cost to Set Cooking Time

A generic microwave supports cooking times for:
```
at least 1 second.
at most 99 minutes and 99 seconds.
```
To set the cooking time, you push at most four digits. The microwave normalizes what you push as four digits by prepending zeroes. It interprets the first two digits as the minutes and the last two digits as the seconds. It then adds them up as the cooking time. For example,
```
You push 9 5 4 (three digits). It is normalized as 0954 and interpreted as 9 minutes and 54 seconds.
You push 0 0 0 8 (four digits). It is interpreted as 0 minutes and 8 seconds.
You push 8 0 9 0. It is interpreted as 80 minutes and 90 seconds.
You push 8 1 3 0. It is interpreted as 81 minutes and 30 seconds.
```
You are given integers startAt, moveCost, pushCost, and targetSeconds. Initially, your finger is on the digit startAt. Moving the finger above any specific digit costs moveCost units of fatigue. Pushing the digit below the finger once costs pushCost units of fatigue.

There can be multiple ways to set the microwave to cook for targetSeconds seconds but you are interested in the way with the minimum cost.

Return the minimum cost to set targetSeconds seconds of cooking time.

Remember that one minute consists of 60 seconds.

 

Example 1:
![img](https://assets.leetcode.com/uploads/2021/12/30/1.png)
```
Input: startAt = 1, moveCost = 2, pushCost = 1, targetSeconds = 600
Output: 6
Explanation: The following are the possible ways to set the cooking time.
- 1 0 0 0, interpreted as 10 minutes and 0 seconds.
  The finger is already on digit 1, pushes 1 (with cost 1), moves to 0 (with cost 2), pushes 0 (with cost 1), pushes 0 (with cost 1), and pushes 0 (with cost 1).
  The cost is: 1 + 2 + 1 + 1 + 1 = 6. This is the minimum cost.
- 0 9 6 0, interpreted as 9 minutes and 60 seconds. That is also 600 seconds.
  The finger moves to 0 (with cost 2), pushes 0 (with cost 1), moves to 9 (with cost 2), pushes 9 (with cost 1), moves to 6 (with cost 2), pushes 6 (with cost 1), moves to 0 (with cost 2), and pushes 0 (with cost 1).
  The cost is: 2 + 1 + 2 + 1 + 2 + 1 + 2 + 1 = 12.
- 9 6 0, normalized as 0960 and interpreted as 9 minutes and 60 seconds.
  The finger moves to 9 (with cost 2), pushes 9 (with cost 1), moves to 6 (with cost 2), pushes 6 (with cost 1), moves to 0 (with cost 2), and pushes 0 (with cost 1).
  The cost is: 2 + 1 + 2 + 1 + 2 + 1 = 9.
```
Example 2:
![img](https://assets.leetcode.com/uploads/2021/12/30/2.png)
```
Input: startAt = 0, moveCost = 1, pushCost = 2, targetSeconds = 76
Output: 6
Explanation: The optimal way is to push two digits: 7 6, interpreted as 76 seconds.
The finger moves to 7 (with cost 1), pushes 7 (with cost 2), moves to 6 (with cost 1), and pushes 6 (with cost 2). The total cost is: 1 + 2 + 1 + 2 = 6
Note other possible ways are 0076, 076, 0116, and 116, but none of them produces the minimum cost.
```

## Solution

Explanation:
```
The maximum possible minutes are: targetSeconds / 60
Check for all possible minutes from 0 to maxmins and the corresponding seconds
cost function returns the cost for given minutes and seconds
moveCost is added to current cost if the finger position is not at the correct number
pushCost is added for each character that is pushed
Maintain the minimum cost and return it
```
Note: We are multiplying mins by 100 to get it into the format of the microwave
Let's say mins = 50,secs = 20
On the microwave we want 5020
For that we can do: 50 * 100 + 20 = 5020



In [174]:
class Solution:
    def minCostSetTime(self, startAt: int, moveCost: int, pushCost: int, targetSeconds: int) -> int:
        def cost(mins, secs):
            s, curr, res = str(mins * 100 + secs), str(startAt), 0
            for ch in s:
                if ch == curr: res += pushCost
                else:
                    res += (pushCost + moveCost)
                    curr = ch
            return res

        maxmins, ans = targetSeconds // 60, float('inf')
        for mins in range(maxmins + 1):
            secs = targetSeconds - mins * 60
            if secs > 99 or mins > 99: continue
            ans = min(ans, cost(mins, secs))
        return ans

# Driver Code
if __name__ == "__main__":
    startAt = 1
    moveCost = 2
    pushCost = 1
    targetSeconds = 600
    print(Solution().minCostSetTime(startAt, moveCost, pushCost, targetSeconds))

6


On further inspection we can deduce that we only really have 2 cases:
```
maxmins, secs
maxmins - 1, secs + 60
```

Since maximum length of characters displayed on microwave = 4, Time Complexity = O(1)

In [175]:
class Solution:
    def minCostSetTime(self, startAt: int, moveCost: int, pushCost: int, targetSeconds: int) -> int:
        def cost(mins, secs):
            if mins > 99 or secs > 99 or mins < 0 or secs < 0: return float('inf')
            s, curr, res = str(mins * 100 + secs), str(startAt), 0
            for ch in s:
                if ch == curr: res += pushCost
                else:
                    res += (pushCost + moveCost)
                    curr = ch
            return res

        mins, secs = targetSeconds // 60, targetSeconds % 60
        return min(cost(mins, secs), cost(mins - 1, secs + 60))

# Driver Code
if __name__ == "__main__":
    startAt = 1
    moveCost = 2
    pushCost = 1
    targetSeconds = 600
    print(Solution().minCostSetTime(startAt, moveCost, pushCost, targetSeconds))


6
